# IMPORTING THE LIBRARIES 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt1 
import peaky

import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 

###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl
from lmfit.models import VoigtModel



from bokeh.io import output_notebook
output_notebook()

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [ ]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store file names
    file_names = []

    # Get a list of .txt files in the folder
    txt_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]

    # Sort the .txt files based on their numerical order
    # txt_files.sort(key=lambda x: int(re.search(r'_(\d+)\.txt', x).group(1)))

    # Read the wavelength values from the first file
    first_file_path = os.path.join(folder_path, txt_files[0])
    wavelength_df = pd.read_csv(first_file_path, header=None, delimiter=';', usecols=[0], names=['wavelength'])

    # Loop through each file in ascending order
    for file_name in txt_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Read intensity values from each file into a DataFrame
        intensity_df = pd.read_csv(file_path, header=None, delimiter=';', usecols=[1], names=['intensity'])
        
        # Store intensity DataFrame
        intensity_dfs.append(intensity_df)
        
        # Store file name
        file_names.append(os.path.splitext(file_name)[0])

    # Concatenate intensity DataFrames
    result_df = pd.concat(intensity_dfs, axis=1)

    # Add the wavelength column to the result DataFrame
    result_df = pd.concat([wavelength_df, result_df], axis=1)

    # Rename the columns with file names
    result_df.columns = ['wavelength'] + file_names

    return result_df


In [ ]:
def average_intensity(df, intensity_name):
    # Select intensity columns (from column 2 to column 13)
    intensity_columns = df.columns[1:]

    # Calculate the mean of intensity columns
    averaged_intensity = df[intensity_columns].mean(axis=1)

    # Create a new DataFrame with wavelength and averaged intensity
    averaged_df = pd.DataFrame({'wavelength': df['wavelength'], intensity_name: averaged_intensity})

    return averaged_df

Loading the dataframe.

In [ ]:
SAMPLE1_Raw_df = load_data('Batch_2/sample_1')
###################################################
SAMPLE2_Raw_df = load_data('Batch_2/sample_2')
###################################################
SAMPLE3_Raw_df = load_data('Batch_2/sample_3')
###################################################
SAMPLE4_Raw_df = load_data('Batch_2/sample_4')
###################################################
SAMPLE5_Raw_df = load_data('Batch_2/sample_5')
###################################################
SAMPLE6_Raw_df = load_data('Batch_2/sample_6')
###################################################
SAMPLE7_Raw_df = load_data('Batch_2/sample_7')
####################################################
SAMPLE8_Raw_df = load_data('Batch_2/sample_8')
####################################################
SAMPLE9_Raw_df = load_data('Batch_3/sample_9')
####################################################
SAMPLE10_Raw_df = load_data('Batch_3/sample_10')
####################################################
SAMPLE11_Raw_df = load_data('Batch_3/sample_11')
####################################################
SAMPLE12_Raw_df = load_data('Batch_3/sample_12')
####################################################
SAMPLE13_Raw_df = load_data('Batch_3/sample_13')
####################################################
SAMPLE14_Raw_df = load_data('Batch_3/sample_14')
####################################################
SAMPLE15_Raw_df = load_data('Batch_3/sample_15')
####################################################
SAMPLE16_Raw_df = load_data('Batch_3/sample_16')
####################################################
SAMPLE17_Raw_df = load_data('Batch_3/sample_17')
####################################################

SAMPLE1_Raw_df

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [ ]:
# peak = 500.606
# peak_min = (peak - 5) 
# peak_max = (peak + 5) 

#Getting the full wavelength
Wavelength_Min = 230 #200.041162 #230
Wavelength_Max = 540 #963.333130 #540


SAMPLE1_Select_df = SAMPLE1_Raw_df[(SAMPLE1_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE1_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE2_Select_df = SAMPLE2_Raw_df[(SAMPLE2_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE2_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE3_Select_df = SAMPLE3_Raw_df[(SAMPLE3_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE3_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE4_Select_df = SAMPLE4_Raw_df[(SAMPLE4_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE4_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE5_Select_df = SAMPLE5_Raw_df[(SAMPLE5_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE5_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE6_Select_df = SAMPLE6_Raw_df[(SAMPLE6_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE6_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE7_Select_df = SAMPLE7_Raw_df[(SAMPLE7_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE7_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE8_Select_df = SAMPLE8_Raw_df[(SAMPLE8_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE8_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE9_Select_df = SAMPLE9_Raw_df[(SAMPLE9_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE9_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE10_Select_df = SAMPLE10_Raw_df[(SAMPLE10_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE10_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE11_Select_df = SAMPLE11_Raw_df[(SAMPLE11_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE11_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE12_Select_df = SAMPLE12_Raw_df[(SAMPLE12_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE12_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE13_Select_df = SAMPLE13_Raw_df[(SAMPLE13_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE13_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE14_Select_df = SAMPLE14_Raw_df[(SAMPLE14_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE14_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE15_Select_df = SAMPLE15_Raw_df[(SAMPLE15_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE15_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE16_Select_df = SAMPLE16_Raw_df[(SAMPLE16_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE16_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE17_Select_df = SAMPLE17_Raw_df[(SAMPLE17_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE17_Raw_df['wavelength'] <= Wavelength_Max)]




SAMPLE1_Select_df


Lets plot a line plot ,to get a better picture 

In [ ]:
Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(SAMPLE1_Select_df.wavelength,SAMPLE1_Select_df.sample1_6 , line_width = 2, color ="blue" )
# #####################################################################################################################
Selected_df_Plot.line(SAMPLE2_Select_df.wavelength,SAMPLE2_Select_df.sample2_6 , line_width = 2, color ="orange" )
# ###############################################################################################################
Selected_df_Plot.line(SAMPLE3_Select_df.wavelength,SAMPLE3_Select_df.sample3_6 , line_width = 2, color ="green")
######################################################################################################################
Selected_df_Plot.line(SAMPLE4_Select_df.wavelength,SAMPLE4_Select_df.sample4_6 , line_width = 2, color ="red")
##############################################################################################################
Selected_df_Plot.line(SAMPLE5_Select_df.wavelength,SAMPLE5_Select_df.sample5_6 , line_width = 2, color ="purple")
###############################################################################################################
Selected_df_Plot.line(SAMPLE6_Select_df.wavelength,SAMPLE6_Select_df.sample6_6 , line_width = 2, color ="brown")
####################################################################################################################
Selected_df_Plot.line(SAMPLE7_Select_df.wavelength,SAMPLE7_Select_df.sample7_6 , line_width = 2, color ="pink")
#################################################################################################################
Selected_df_Plot.line(SAMPLE8_Select_df.wavelength,SAMPLE8_Select_df.sample8_6 , line_width = 2, color ="gray")
# ###################################################################################################################
Selected_df_Plot.line(SAMPLE9_Select_df.wavelength,SAMPLE9_Select_df.sample9_6 , line_width = 2, color ="olive")
# ###################################################################################################################
Selected_df_Plot.line(SAMPLE10_Select_df.wavelength,SAMPLE10_Select_df.sample10_6 , line_width = 2, color ="Cyan")
###################################################################################################################
Selected_df_Plot.line(SAMPLE11_Select_df.wavelength,SAMPLE11_Select_df.sample11_6 , line_width = 2, color ="black")
###################################################################################################################
Selected_df_Plot.line(SAMPLE12_Select_df.wavelength,SAMPLE12_Select_df.sample12_6 , line_width = 2, color ="tomato")
###################################################################################################################
Selected_df_Plot.line(SAMPLE13_Select_df.wavelength,SAMPLE13_Select_df.sample13_6 , line_width = 2, color ="steelblue")
###################################################################################################################
Selected_df_Plot.line(SAMPLE14_Select_df.wavelength,SAMPLE14_Select_df.sample14_6 , line_width = 2, color ="limegreen")
###################################################################################################################
Selected_df_Plot.line(SAMPLE15_Select_df.wavelength,SAMPLE15_Select_df.sample15_6 , line_width = 2, color ="deeppink")
###################################################################################################################
Selected_df_Plot.line(SAMPLE16_Select_df.wavelength,SAMPLE16_Select_df.sample16_6 , line_width = 2, color ="gold")
###################################################################################################################
Selected_df_Plot.line(SAMPLE17_Select_df.wavelength,SAMPLE17_Select_df.sample17_6 , line_width = 2, color ="indigo")
# ###################################################################################################################

Selected_df_Plot.width = 900
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.

In [ ]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.asls(df[col])
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

In [ ]:
SAMPLE1_BaselineCorrected_df = baseline_correction(SAMPLE1_Select_df)
SAMPLE2_BaselineCorrected_df = baseline_correction(SAMPLE2_Select_df)
SAMPLE3_BaselineCorrected_df = baseline_correction(SAMPLE3_Select_df)
SAMPLE4_BaselineCorrected_df = baseline_correction(SAMPLE4_Select_df)
SAMPLE5_BaselineCorrected_df = baseline_correction(SAMPLE5_Select_df)
SAMPLE6_BaselineCorrected_df = baseline_correction(SAMPLE6_Select_df)
SAMPLE7_BaselineCorrected_df = baseline_correction(SAMPLE7_Select_df)
SAMPLE8_BaselineCorrected_df = baseline_correction(SAMPLE8_Select_df)
SAMPLE9_BaselineCorrected_df = baseline_correction(SAMPLE9_Select_df)
SAMPLE10_BaselineCorrected_df = baseline_correction(SAMPLE10_Select_df)
SAMPLE11_BaselineCorrected_df = baseline_correction(SAMPLE11_Select_df)
SAMPLE12_BaselineCorrected_df = baseline_correction(SAMPLE12_Select_df)
SAMPLE13_BaselineCorrected_df = baseline_correction(SAMPLE13_Select_df)
SAMPLE14_BaselineCorrected_df = baseline_correction(SAMPLE14_Select_df)
SAMPLE15_BaselineCorrected_df = baseline_correction(SAMPLE15_Select_df)
SAMPLE16_BaselineCorrected_df = baseline_correction(SAMPLE16_Select_df)
SAMPLE17_BaselineCorrected_df = baseline_correction(SAMPLE17_Select_df)

SAMPLE1_BaselineCorrected_df

In [ ]:
SAMPLE1_BaselineCorrected_all_df = baseline_correction(SAMPLE1_Raw_df)
SAMPLE2_BaselineCorrected_all_df = baseline_correction(SAMPLE2_Raw_df)
SAMPLE3_BaselineCorrected_all_df = baseline_correction(SAMPLE3_Raw_df)
SAMPLE4_BaselineCorrected_all_df = baseline_correction(SAMPLE4_Raw_df)
SAMPLE5_BaselineCorrected_all_df = baseline_correction(SAMPLE5_Raw_df)
SAMPLE6_BaselineCorrected_all_df = baseline_correction(SAMPLE6_Raw_df)
SAMPLE7_BaselineCorrected_all_df = baseline_correction(SAMPLE7_Raw_df)
SAMPLE8_BaselineCorrected_all_df = baseline_correction(SAMPLE8_Raw_df)
SAMPLE9_BaselineCorrected_all_df = baseline_correction(SAMPLE9_Raw_df)
SAMPLE10_BaselineCorrected_all_df = baseline_correction(SAMPLE10_Raw_df)
SAMPLE11_BaselineCorrected_all_df = baseline_correction(SAMPLE11_Raw_df)
SAMPLE12_BaselineCorrected_all_df = baseline_correction(SAMPLE12_Raw_df)
SAMPLE13_BaselineCorrected_all_df = baseline_correction(SAMPLE13_Raw_df)
SAMPLE14_BaselineCorrected_all_df = baseline_correction(SAMPLE14_Raw_df)
SAMPLE15_BaselineCorrected_all_df = baseline_correction(SAMPLE15_Raw_df)
SAMPLE16_BaselineCorrected_all_df = baseline_correction(SAMPLE16_Raw_df)
SAMPLE17_BaselineCorrected_all_df = baseline_correction(SAMPLE17_Raw_df)

SAMPLE1_BaselineCorrected_all_df

In [ ]:
Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Baseline_Correction_Plot.line(SAMPLE9_Select_df.wavelength,SAMPLE9_Select_df.sample9_6 , line_width = 2, color ="red" )
Baseline_Correction_Plot.line(SAMPLE9_BaselineCorrected_df.wavelength,SAMPLE9_BaselineCorrected_df.sample9_6 , line_width = 2, color ="green" )


Baseline_Correction_Plot.width = 900
Baseline_Correction_Plot.height = 500
show(Baseline_Correction_Plot)

2) Normalization - Its usually noticed that , for a measurement of a similar sample ,there is a lot of scattering in the intensities , this typical artifect is called scattering.

For Instance , you could see the plot above , Though this are the plots from the same sample SAMPLE1 , measured on 12 different Areas , its quite visible , for some  spectras , the peak heights or Intensities are not same.


In [ ]:

# def standard_normal_variate_normalization(df):
#     # Selecting only the intensity columns (assuming the first column is wavelength)
#     intensities = df.iloc[:, 1:]

#     # Calculating mean and standard deviation for each column
#     means = intensities.mean(axis=0)
#     stds = intensities.std(axis=0)

#     # Applying standard normal variate normalization column-wise
#     normalized_intensities = (intensities - means) / stds

#     # Combining the wavelength column with normalized intensities
#     normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

#     # Calculating mean and standard deviation for each normalized column
#     normalized_means = normalized_intensities.mean(axis=0)
#     normalized_stds = normalized_intensities.std(axis=0)

#     return normalized_df



In [ ]:
def standard_normal_variate_normalization(df):
    # Select columns containing intensities
    intensity_cols = df.columns[1:]  #  columns 2 to 13 are intensities and are stored in intensity_cols

    # SVN normalization
    for col in intensity_cols:
        mean_intensity = df[col].mean()
        std_intensity = df[col].std()
        df[col] = (df[col] - mean_intensity) / std_intensity

    return df

In [ ]:

def total_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column in intensities.columns:
        spectrum = intensities[column]
        total_intensity = spectrum.sum()
        
        # Normalize the spectrum by dividing each intensity value by the total intensity
        normalized_spectrum = spectrum / total_intensity
        
        # Append the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities[column] = normalized_spectrum

    # Combine the wavelength column with normalized intensities
    normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

    return normalized_df

In [ ]:
def unit_norm_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_1 = df.iloc[:, 1:]  
    
    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_1 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_1 in intensities_1.columns:
        spectrum_1 = intensities_1[column_1]
        
        # Calculate the norm of the spectrum using a simple equation
        spectrum_norm = np.sqrt(np.sum(spectrum_1 ** 2))
        
        # Normalize the spectrum by dividing each intensity value by its norm
        normalized_spectrum_1 = spectrum_1 / spectrum_norm
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_1[column_1] = normalized_spectrum_1

    # Combine the wavelength column with normalized intensities
    normalized_df_1 = pd.concat([df.iloc[:, 0], normalized_intensities_1], axis=1)
    
    return normalized_df_1

In [ ]:
def max_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_2 = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_2 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_2 in intensities_2.columns:
        spectrum_2 = intensities_2[column_2]
        
        # Find the maximum intensity value in the spectrum
        max_intensity = spectrum_2.max()
        
        # Normalize the spectrum by dividing each intensity value by the maximum intensity
        normalized_spectrum_2 = spectrum_2 / max_intensity
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_2[column_2] = normalized_spectrum_2

    # Combine the wavelength column with normalized intensities
    normalized_df_2 = pd.concat([df.iloc[:, 0], normalized_intensities_2], axis=1)

    return normalized_df_2

In [ ]:
SAMPLE1_Normalized_df = total_intensity_normalization(SAMPLE1_BaselineCorrected_df)
SAMPLE2_Normalized_df = total_intensity_normalization(SAMPLE2_BaselineCorrected_df)
SAMPLE3_Normalized_df = total_intensity_normalization(SAMPLE3_BaselineCorrected_df)
SAMPLE4_Normalized_df = total_intensity_normalization(SAMPLE4_BaselineCorrected_df)
SAMPLE5_Normalized_df = total_intensity_normalization(SAMPLE5_BaselineCorrected_df)
SAMPLE6_Normalized_df = total_intensity_normalization(SAMPLE6_BaselineCorrected_df)
SAMPLE7_Normalized_df = total_intensity_normalization(SAMPLE7_BaselineCorrected_df)
SAMPLE8_Normalized_df = total_intensity_normalization(SAMPLE8_BaselineCorrected_df)
SAMPLE9_Normalized_df = total_intensity_normalization(SAMPLE9_BaselineCorrected_df)
SAMPLE10_Normalized_df = total_intensity_normalization(SAMPLE10_BaselineCorrected_df)
SAMPLE11_Normalized_df = total_intensity_normalization(SAMPLE11_BaselineCorrected_df)
SAMPLE12_Normalized_df = total_intensity_normalization(SAMPLE12_BaselineCorrected_df)
SAMPLE13_Normalized_df = total_intensity_normalization(SAMPLE13_BaselineCorrected_df)
SAMPLE14_Normalized_df = total_intensity_normalization(SAMPLE14_BaselineCorrected_df)
SAMPLE15_Normalized_df = total_intensity_normalization(SAMPLE15_BaselineCorrected_df)
SAMPLE16_Normalized_df = total_intensity_normalization(SAMPLE16_BaselineCorrected_df)
SAMPLE17_Normalized_df = total_intensity_normalization(SAMPLE17_BaselineCorrected_df)

SAMPLE1_Normalized_df

Normalized_Plot = figure(title = 'After Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_1 , line_width =2 , color = "red")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_2 , line_width =2 , color = "green")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_3 , line_width =2 , color = "pink")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_4 , line_width =2 , color = "yellow")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_5 , line_width =2 , color = "blue")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_6 , line_width =2 , color = "orange")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_7, line_width =2 , color = "brown")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_8 , line_width =2 , color = "black")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_9 , line_width =2 , color = "tomato")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_10 , line_width =2 , color = "steelblue")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_11, line_width =2 , color = "gold")
Normalized_Plot.line(SAMPLE1_Normalized_df.wavelength,SAMPLE1_Normalized_df.sample1_12 , line_width =2 , color = "indigo")




Normalized_Plot.width = 900
Normalized_Plot.height = 500
show(Normalized_Plot)



# mean_intensity = SAMPLE1_Normalized_df['sample1_13'].mean()
# std_intensity = SAMPLE1_Normalized_df['sample1_13'].std()

# print(mean_intensity)
# print(std_intensity)

In [ ]:
SAMPLE1_Normalized_all_df = total_intensity_normalization(SAMPLE1_BaselineCorrected_all_df)
SAMPLE2_Normalized_all_df = total_intensity_normalization(SAMPLE2_BaselineCorrected_all_df)
SAMPLE3_Normalized_all_df = total_intensity_normalization(SAMPLE3_BaselineCorrected_all_df)
SAMPLE4_Normalized_all_df = total_intensity_normalization(SAMPLE4_BaselineCorrected_all_df)
SAMPLE5_Normalized_all_df = total_intensity_normalization(SAMPLE5_BaselineCorrected_all_df)
SAMPLE6_Normalized_all_df = total_intensity_normalization(SAMPLE6_BaselineCorrected_all_df)
SAMPLE7_Normalized_all_df = total_intensity_normalization(SAMPLE7_BaselineCorrected_all_df)
SAMPLE8_Normalized_all_df = total_intensity_normalization(SAMPLE8_BaselineCorrected_all_df)
SAMPLE9_Normalized_all_df = total_intensity_normalization(SAMPLE9_BaselineCorrected_all_df)
SAMPLE10_Normalized_all_df = total_intensity_normalization(SAMPLE10_BaselineCorrected_all_df)
SAMPLE11_Normalized_all_df = total_intensity_normalization(SAMPLE11_BaselineCorrected_all_df)
SAMPLE12_Normalized_all_df = total_intensity_normalization(SAMPLE12_BaselineCorrected_all_df)
SAMPLE13_Normalized_all_df = total_intensity_normalization(SAMPLE13_BaselineCorrected_all_df)
SAMPLE14_Normalized_all_df = total_intensity_normalization(SAMPLE14_BaselineCorrected_all_df)
SAMPLE15_Normalized_all_df = total_intensity_normalization(SAMPLE15_BaselineCorrected_all_df)
SAMPLE16_Normalized_all_df = total_intensity_normalization(SAMPLE16_BaselineCorrected_all_df)
SAMPLE17_Normalized_all_df = total_intensity_normalization(SAMPLE17_BaselineCorrected_all_df)

print(SAMPLE1_Normalized_all_df)

3) Smoothing of the Spectrum - Now even though , the signal from the hLIBS Instrument doesn't appear to have much noise , we could still implement this additional signal smoothing steps , in our Data Processing Pipeline

In [ ]:
def apply_savitzky_golay_smoothing(df, window_length=20, polyorder=15):
    # Selecting only the normalized intensity columns
    normalized_intensities = df.iloc[:, 1:]

    # Applying Savitzky-Golay smoothing to each intensity column
    smoothed_intensities = normalized_intensities.apply(lambda x: savgol_filter(x, window_length, polyorder), axis=0)

    # Combining wavelength column with smoothed intensities
    smoothed_df = pd.concat([df.iloc[:, 0], smoothed_intensities], axis=1)

    return smoothed_df

In [ ]:
SAMPLE1_Smoothed_df = SAMPLE1_Normalized_df
SAMPLE2_Smoothed_df = SAMPLE2_Normalized_df
SAMPLE3_Smoothed_df = SAMPLE3_Normalized_df
SAMPLE4_Smoothed_df = SAMPLE4_Normalized_df
SAMPLE5_Smoothed_df = SAMPLE5_Normalized_df
SAMPLE6_Smoothed_df = SAMPLE6_Normalized_df
SAMPLE7_Smoothed_df = SAMPLE7_Normalized_df
SAMPLE8_Smoothed_df = SAMPLE8_Normalized_df
SAMPLE9_Smoothed_df = SAMPLE9_Normalized_df
SAMPLE10_Smoothed_df = SAMPLE10_Normalized_df
SAMPLE11_Smoothed_df = SAMPLE11_Normalized_df
SAMPLE12_Smoothed_df = SAMPLE12_Normalized_df
SAMPLE13_Smoothed_df = SAMPLE13_Normalized_df
SAMPLE14_Smoothed_df = SAMPLE14_Normalized_df
SAMPLE15_Smoothed_df = SAMPLE15_Normalized_df
SAMPLE16_Smoothed_df = SAMPLE16_Normalized_df
SAMPLE17_Smoothed_df = SAMPLE17_Normalized_df


Smoothed_Plot = figure(title = 'After Smoothing of the Spectrum' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Smoothed_Plot.line(SAMPLE1_Smoothed_df.wavelength,SAMPLE1_Smoothed_df.sample1_6 , line_width = 2, color ="blue" )
#####################################################################################################################
Smoothed_Plot.line(SAMPLE2_Smoothed_df.wavelength,SAMPLE2_Smoothed_df.sample2_6 , line_width = 2, color ="orange" )
###############################################################################################################
Smoothed_Plot.line(SAMPLE3_Smoothed_df.wavelength,SAMPLE3_Smoothed_df.sample3_6 , line_width = 2, color ="green")
######################################################################################################################
Smoothed_Plot.line(SAMPLE4_Smoothed_df.wavelength,SAMPLE4_Smoothed_df.sample4_6 , line_width = 2, color ="red")
#############################################################################################################
Smoothed_Plot.line(SAMPLE5_Smoothed_df.wavelength,SAMPLE5_Smoothed_df.sample5_6 , line_width = 2, color ="purple")
###############################################################################################################
Smoothed_Plot.line(SAMPLE6_Smoothed_df.wavelength,SAMPLE6_Smoothed_df.sample6_6 , line_width = 2, color ="brown")
###################################################################################################################
Smoothed_Plot.line(SAMPLE7_Smoothed_df.wavelength,SAMPLE7_Smoothed_df.sample7_6 , line_width = 2, color ="pink")
##################################################################################################################
Smoothed_Plot.line(SAMPLE8_Smoothed_df.wavelength,SAMPLE8_Smoothed_df.sample8_6 , line_width = 2, color ="gray")
# ###################################################################################################################
Smoothed_Plot.line(SAMPLE9_Smoothed_df.wavelength,SAMPLE9_Smoothed_df.sample9_6 , line_width = 2, color ="olive")
###################################################################################################################
Smoothed_Plot.line(SAMPLE10_Smoothed_df.wavelength,SAMPLE10_Smoothed_df.sample10_6 , line_width = 2, color ="Cyan")
###################################################################################################################
Smoothed_Plot.line(SAMPLE11_Smoothed_df.wavelength,SAMPLE11_Smoothed_df.sample11_6 , line_width = 2, color ="black")
###################################################################################################################
Smoothed_Plot.line(SAMPLE12_Smoothed_df.wavelength,SAMPLE12_Smoothed_df.sample12_6 , line_width = 2, color ="tomato")
###################################################################################################################
Smoothed_Plot.line(SAMPLE13_Smoothed_df.wavelength,SAMPLE13_Smoothed_df.sample13_6 , line_width = 2, color ="steelblue")
###################################################################################################################
Smoothed_Plot.line(SAMPLE14_Smoothed_df.wavelength,SAMPLE14_Smoothed_df.sample14_6 , line_width = 2, color ="limegreen")
###################################################################################################################
Smoothed_Plot.line(SAMPLE15_Smoothed_df.wavelength,SAMPLE15_Smoothed_df.sample15_6 , line_width = 2, color ="deeppink")
###################################################################################################################
Smoothed_Plot.line(SAMPLE16_Smoothed_df.wavelength,SAMPLE16_Smoothed_df.sample16_6 , line_width = 2, color ="gold")
###################################################################################################################
Smoothed_Plot.line(SAMPLE17_Smoothed_df.wavelength,SAMPLE17_Smoothed_df.sample17_6 , line_width = 2, color ="indigo")
###################################################################################################################


Smoothed_Plot.width = 900
Smoothed_Plot.height = 500
#show(Smoothed_Plot)

SAMPLE1_Smoothed_df

4) Averaging the Spectrum - The final steps is to Average the specta files

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

# Define the Gaussian profile
def gaussian(x, center, amplitude, sigma):
    return amplitude * np.exp(-((x - center)**2) / (2 * sigma**2))

def fit_gaussian_profile(df):
    # Extract the wavelength data
    wavelength = df.iloc[:, 0].values

    # Prepare to store the fitted profiles
    fitted_data = pd.DataFrame({'wavelength': wavelength})

    # Iterate over each intensity column
    for column in df.columns[1:]:
        intensity = df[column].values

        # Find the peak
        peak_indices, _ = find_peaks(intensity)
        if len(peak_indices) == 0:
            raise ValueError(f"No peaks found in the data for column {column}.")
        
        peak_index = peak_indices[np.argmax(intensity[peak_indices])]
        center_guess = wavelength[peak_index]
        amplitude_guess = intensity[peak_index]

        # Find FWHM
        half_max = amplitude_guess / 2
        indices_above_half_max = np.where(intensity >= half_max)[0]

        if len(indices_above_half_max) >= 2:
            # Interpolate to find the exact FWHM
            left_index = indices_above_half_max[0]
            right_index = indices_above_half_max[-1]
            fwhm = wavelength[right_index] - wavelength[left_index]

            # Convert FWHM to sigma
            sigma_guess = fwhm / (2 * np.sqrt(2 * np.log(2)))
        else:
            # Fallback to the range-based estimate if FWHM calculation fails
            sigma_guess = (wavelength[-1] - wavelength[0]) / 10

        # Initial guesses for the fit parameters
        initial_guesses = [center_guess, amplitude_guess, sigma_guess]

        # Fit the Gaussian profile to the data
        popt, _ = curve_fit(gaussian, wavelength, intensity, p0=initial_guesses)

        # Generate the fitted Gaussian profile using the best fit parameters
        fitted_profile = gaussian(wavelength, *popt)

        # Add the fitted profile to the DataFrame
        fitted_data[column] = fitted_profile

    return fitted_data


In [ ]:
# import numpy as np
# import pandas as pd
# from scipy.optimize import curve_fit
# from scipy.signal import find_peaks
# from scipy.special import wofz

# # Define the Voigt profile
# def voigt(x, center, amplitude, sigma, gamma):
#     z = ((x - center) + 1j * gamma) / (sigma * np.sqrt(2))
#     return amplitude * np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))

# def fit_voigt_profile(df):
#     # Extract the wavelength data
#     wavelength = df.iloc[:, 0].values

#     # Prepare to store the fitted profiles
#     fitted_data = pd.DataFrame({'wavelength': wavelength})

#     # Iterate over each intensity column
#     for column in df.columns[1:]:
#         intensity = df[column].values

#         # Find the peak
#         peak_indices, _ = find_peaks(intensity)
#         if len(peak_indices) == 0:
#             raise ValueError(f"No peaks found in the data for column {column}.")
        
#         peak_index = peak_indices[np.argmax(intensity[peak_indices])]
#         center_guess = wavelength[peak_index]
#         amplitude_guess = intensity[peak_index]

#         # Find FWHM
#         half_max = amplitude_guess / 2
#         indices_above_half_max = np.where(intensity >= half_max)[0]

#         if len(indices_above_half_max) >= 2:
#             # Interpolate to find the exact FWHM
#             left_index = indices_above_half_max[0]
#             right_index = indices_above_half_max[-1]
#             fwhm = wavelength[right_index] - wavelength[left_index]

#             # Convert FWHM to sigma (for Gaussian part of Voigt)
#             sigma_guess = fwhm / (2 * np.sqrt(2 * np.log(2)))
#         else:
#             # Fallback to the range-based estimate if FWHM calculation fails
#             sigma_guess = (wavelength[-1] - wavelength[0]) / 10

#         # Estimate for gamma (Lorentzian part of Voigt)
#         gamma_guess = sigma_guess / 2

#         # Initial guesses for the fit parameters
#         initial_guesses = [center_guess, amplitude_guess, sigma_guess, gamma_guess]

#         # Fit the Voigt profile to the data
#         popt, _ = curve_fit(voigt, wavelength, intensity, p0=initial_guesses)

#         # Generate the fitted Voigt profile using the best fit parameters
#         fitted_profile = voigt(wavelength, *popt)

#         # Add the fitted profile to the DataFrame
#         fitted_data[column] = fitted_profile

#     return fitted_data


In [ ]:
def average_intensity(df, intensity_name):
    # Select intensity columns (from column 2 to column 13)
    intensity_columns = df.columns[1:]

    # Calculate the mean of intensity columns
    averaged_intensity = df[intensity_columns].mean(axis=1)

    # Create a new DataFrame with wavelength and averaged intensity
    averaged_df = pd.DataFrame({'wavelength': df['wavelength'], intensity_name: averaged_intensity})

    return averaged_df

In [ ]:
SAMPLE1_Averaged_df = average_intensity(df = SAMPLE1_Smoothed_df , intensity_name= 'sample1')
SAMPLE2_Averaged_df = average_intensity(df = SAMPLE2_Smoothed_df , intensity_name= 'sample2')
SAMPLE3_Averaged_df = average_intensity(df = SAMPLE3_Smoothed_df , intensity_name= 'sample3')
SAMPLE4_Averaged_df = average_intensity(df = SAMPLE4_Smoothed_df , intensity_name= 'sample4')
SAMPLE5_Averaged_df = average_intensity(df = SAMPLE5_Smoothed_df , intensity_name= 'sample5')
SAMPLE6_Averaged_df = average_intensity(df = SAMPLE6_Smoothed_df , intensity_name= 'sample6')
SAMPLE7_Averaged_df = average_intensity(df = SAMPLE7_Smoothed_df , intensity_name= 'sample7')
SAMPLE8_Averaged_df = average_intensity(df = SAMPLE8_Smoothed_df , intensity_name= 'sample8')
SAMPLE9_Averaged_df = average_intensity(df = SAMPLE9_Smoothed_df , intensity_name= 'sample9')
SAMPLE10_Averaged_df = average_intensity(df = SAMPLE10_Smoothed_df , intensity_name= 'sample10')
SAMPLE11_Averaged_df = average_intensity(df = SAMPLE11_Smoothed_df , intensity_name= 'sample11')
SAMPLE12_Averaged_df = average_intensity(df = SAMPLE12_Smoothed_df , intensity_name= 'sample12')
SAMPLE13_Averaged_df = average_intensity(df = SAMPLE13_Smoothed_df , intensity_name= 'sample13')
SAMPLE14_Averaged_df = average_intensity(df = SAMPLE14_Smoothed_df , intensity_name= 'sample14')
SAMPLE15_Averaged_df = average_intensity(df = SAMPLE15_Smoothed_df , intensity_name= 'sample15')
SAMPLE16_Averaged_df = average_intensity(df = SAMPLE16_Smoothed_df , intensity_name= 'sample16')
SAMPLE17_Averaged_df = average_intensity(df = SAMPLE17_Smoothed_df , intensity_name= 'sample17')

print(SAMPLE1_Averaged_df)

In [ ]:
SAMPLE1_Averaged_full_df = average_intensity(df = SAMPLE1_Normalized_all_df , intensity_name= 'sample1')
SAMPLE2_Averaged_full_df = average_intensity(df = SAMPLE2_Normalized_all_df , intensity_name= 'sample2')
SAMPLE3_Averaged_full_df = average_intensity(df = SAMPLE3_Normalized_all_df , intensity_name= 'sample3')
SAMPLE4_Averaged_full_df = average_intensity(df = SAMPLE4_Normalized_all_df , intensity_name= 'sample4')
SAMPLE5_Averaged_full_df = average_intensity(df = SAMPLE5_Normalized_all_df , intensity_name= 'sample5')
SAMPLE6_Averaged_full_df = average_intensity(df = SAMPLE6_Normalized_all_df , intensity_name= 'sample6')
SAMPLE7_Averaged_full_df = average_intensity(df = SAMPLE7_Normalized_all_df , intensity_name= 'sample7')
SAMPLE8_Averaged_full_df = average_intensity(df = SAMPLE8_Normalized_all_df , intensity_name= 'sample8')
SAMPLE9_Averaged_full_df = average_intensity(df = SAMPLE9_Normalized_all_df , intensity_name= 'sample9')
SAMPLE10_Averaged_full_df = average_intensity(df = SAMPLE10_Normalized_all_df , intensity_name= 'sample10')
SAMPLE11_Averaged_full_df = average_intensity(df = SAMPLE11_Normalized_all_df , intensity_name= 'sample11')
SAMPLE12_Averaged_full_df = average_intensity(df = SAMPLE12_Normalized_all_df , intensity_name= 'sample12')
SAMPLE13_Averaged_full_df = average_intensity(df = SAMPLE13_Normalized_all_df , intensity_name= 'sample13')
SAMPLE14_Averaged_full_df = average_intensity(df = SAMPLE14_Normalized_all_df , intensity_name= 'sample14')
SAMPLE15_Averaged_full_df = average_intensity(df = SAMPLE15_Normalized_all_df , intensity_name= 'sample15')
SAMPLE16_Averaged_full_df = average_intensity(df = SAMPLE16_Normalized_all_df , intensity_name= 'sample16')
SAMPLE17_Averaged_full_df = average_intensity(df = SAMPLE17_Normalized_all_df , intensity_name= 'sample17')

print(SAMPLE1_Averaged_full_df)

In [ ]:
Averaged_df_Plot = figure(title = 'Averaged Spectras' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Averaged_df_Plot.line(SAMPLE1_Averaged_df.wavelength,SAMPLE1_Averaged_df.sample1 , line_width = 2, color ="blue" )
#####################################################################################################################
Averaged_df_Plot.line(SAMPLE2_Averaged_df.wavelength,SAMPLE2_Averaged_df.sample2 , line_width = 2, color ="orange" )
###############################################################################################################
Averaged_df_Plot.line(SAMPLE3_Averaged_df.wavelength,SAMPLE3_Averaged_df.sample3 , line_width = 2, color ="green")
######################################################################################################################
Averaged_df_Plot.line(SAMPLE4_Averaged_df.wavelength,SAMPLE4_Averaged_df.sample4 , line_width = 2, color ="red")
##############################################################################################################
Averaged_df_Plot.line(SAMPLE5_Averaged_df.wavelength,SAMPLE5_Averaged_df.sample5 , line_width = 2, color ="purple")
###############################################################################################################
Averaged_df_Plot.line(SAMPLE6_Averaged_df.wavelength,SAMPLE6_Averaged_df.sample6 , line_width = 2, color ="brown")
####################################################################################################################
Averaged_df_Plot.line(SAMPLE7_Averaged_df.wavelength,SAMPLE7_Averaged_df.sample7 , line_width = 2, color ="pink")
##################################################################################################################
Averaged_df_Plot.line(SAMPLE8_Averaged_df.wavelength,SAMPLE8_Averaged_df.sample8 , line_width = 2, color ="gray")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE9_Averaged_df.wavelength,SAMPLE9_Averaged_df.sample9 , line_width = 2, color ="olive")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE10_Averaged_df.wavelength,SAMPLE10_Averaged_df.sample10 , line_width = 2, color ="Cyan")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE11_Averaged_df.wavelength,SAMPLE11_Averaged_df.sample11 , line_width = 2, color ="black")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE12_Averaged_df.wavelength,SAMPLE12_Averaged_df.sample12 , line_width = 2, color ="tomato")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE13_Averaged_df.wavelength,SAMPLE13_Averaged_df.sample13 , line_width = 2, color ="steelblue")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE14_Averaged_df.wavelength,SAMPLE14_Averaged_df.sample14 , line_width = 2, color ="limegreen")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE15_Averaged_df.wavelength,SAMPLE15_Averaged_df.sample15 , line_width = 2, color ="deeppink")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE16_Averaged_df.wavelength,SAMPLE16_Averaged_df.sample16 , line_width = 2, color ="gold")
###################################################################################################################
Averaged_df_Plot.line(SAMPLE17_Averaged_df.wavelength,SAMPLE17_Averaged_df.sample17 , line_width = 2, color ="indigo")
###################################################################################################################

Averaged_df_Plot.width = 900
Averaged_df_Plot.height = 500
show(Averaged_df_Plot)

# Selection of ROI 

Now , as we can see , we have datapoints from wavelength , in which we defined above , but our ROI is just one peak.

In [ ]:
# For getting all datapoints 

Peak_Pos = '403.44'

Peak_Min = 403.404
Peak_Max = 403.479

Element_name = 'Mn'

SAMPLE1_Processed_Selectall_df = SAMPLE1_Smoothed_df[(SAMPLE1_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE1_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE2_Processed_Selectall_df = SAMPLE2_Smoothed_df[(SAMPLE2_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE2_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE3_Processed_Selectall_df = SAMPLE3_Smoothed_df[(SAMPLE3_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE3_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE4_Processed_Selectall_df = SAMPLE4_Smoothed_df[(SAMPLE4_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE4_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE5_Processed_Selectall_df = SAMPLE5_Smoothed_df[(SAMPLE5_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE5_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE6_Processed_Selectall_df = SAMPLE6_Smoothed_df[(SAMPLE6_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE6_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE7_Processed_Selectall_df = SAMPLE7_Smoothed_df[(SAMPLE7_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE7_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE8_Processed_Selectall_df = SAMPLE8_Smoothed_df[(SAMPLE8_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE8_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE9_Processed_Selectall_df = SAMPLE9_Smoothed_df[(SAMPLE9_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE9_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE10_Processed_Selectall_df = SAMPLE10_Smoothed_df[(SAMPLE10_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE10_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE11_Processed_Selectall_df = SAMPLE11_Smoothed_df[(SAMPLE11_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE11_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE12_Processed_Selectall_df = SAMPLE12_Smoothed_df[(SAMPLE12_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE12_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE13_Processed_Selectall_df = SAMPLE13_Smoothed_df[(SAMPLE13_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE13_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE14_Processed_Selectall_df = SAMPLE14_Smoothed_df[(SAMPLE14_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE14_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE15_Processed_Selectall_df = SAMPLE15_Smoothed_df[(SAMPLE15_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE15_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE16_Processed_Selectall_df = SAMPLE16_Smoothed_df[(SAMPLE16_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE16_Smoothed_df['wavelength'] <= Peak_Max)]
SAMPLE17_Processed_Selectall_df = SAMPLE17_Smoothed_df[(SAMPLE17_Smoothed_df['wavelength'] >= Peak_Min) & (SAMPLE17_Smoothed_df['wavelength'] <= Peak_Max)]

SAMPLE1_Processed_Selectall_df

In [ ]:
#GAUSSIAN FITTING

SAMPLE1_Fitted_Allpoints = fit_gaussian_profile(SAMPLE1_Processed_Selectall_df)
SAMPLE2_Fitted_Allpoints = fit_gaussian_profile(SAMPLE2_Processed_Selectall_df)
SAMPLE3_Fitted_Allpoints = fit_gaussian_profile(SAMPLE3_Processed_Selectall_df)
SAMPLE4_Fitted_Allpoints = fit_gaussian_profile(SAMPLE4_Processed_Selectall_df)
SAMPLE5_Fitted_Allpoints = fit_gaussian_profile(SAMPLE5_Processed_Selectall_df)
SAMPLE6_Fitted_Allpoints = fit_gaussian_profile(SAMPLE6_Processed_Selectall_df)
SAMPLE7_Fitted_Allpoints = (SAMPLE7_Processed_Selectall_df)
SAMPLE8_Fitted_Allpoints = fit_gaussian_profile(SAMPLE8_Processed_Selectall_df)
SAMPLE9_Fitted_Allpoints = fit_gaussian_profile(SAMPLE9_Processed_Selectall_df)
SAMPLE10_Fitted_Allpoints = fit_gaussian_profile(SAMPLE10_Processed_Selectall_df)
SAMPLE11_Fitted_Allpoints = fit_gaussian_profile(SAMPLE11_Processed_Selectall_df)
SAMPLE12_Fitted_Allpoints = fit_gaussian_profile(SAMPLE12_Processed_Selectall_df)
SAMPLE13_Fitted_Allpoints = fit_gaussian_profile(SAMPLE13_Processed_Selectall_df)
SAMPLE14_Fitted_Allpoints = fit_gaussian_profile(SAMPLE14_Processed_Selectall_df)
SAMPLE15_Fitted_Allpoints = fit_gaussian_profile(SAMPLE15_Processed_Selectall_df)
SAMPLE16_Fitted_Allpoints = fit_gaussian_profile(SAMPLE16_Processed_Selectall_df)
SAMPLE17_Fitted_Allpoints = (SAMPLE17_Processed_Selectall_df)

SAMPLE1_Fitted_Allpoints

In [ ]:
SAMPLE1_Processed_Selectaverage_df = SAMPLE1_Averaged_df[(SAMPLE1_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE1_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE2_Processed_Selectaverage_df = SAMPLE2_Averaged_df[(SAMPLE2_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE2_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE3_Processed_Selectaverage_df = SAMPLE3_Averaged_df[(SAMPLE3_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE3_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE4_Processed_Selectaverage_df = SAMPLE4_Averaged_df[(SAMPLE4_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE4_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE5_Processed_Selectaverage_df = SAMPLE5_Averaged_df[(SAMPLE5_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE5_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE6_Processed_Selectaverage_df = SAMPLE6_Averaged_df[(SAMPLE6_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE6_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE7_Processed_Selectaverage_df = SAMPLE7_Averaged_df[(SAMPLE7_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE7_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE8_Processed_Selectaverage_df = SAMPLE8_Averaged_df[(SAMPLE8_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE8_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE9_Processed_Selectaverage_df = SAMPLE9_Averaged_df[(SAMPLE9_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE9_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE10_Processed_Selectaverage_df = SAMPLE10_Averaged_df[(SAMPLE10_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE10_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE11_Processed_Selectaverage_df = SAMPLE11_Averaged_df[(SAMPLE11_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE11_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE12_Processed_Selectaverage_df = SAMPLE12_Averaged_df[(SAMPLE12_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE12_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE13_Processed_Selectaverage_df = SAMPLE13_Averaged_df[(SAMPLE13_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE13_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE14_Processed_Selectaverage_df = SAMPLE14_Averaged_df[(SAMPLE14_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE14_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE15_Processed_Selectaverage_df = SAMPLE15_Averaged_df[(SAMPLE15_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE15_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE16_Processed_Selectaverage_df = SAMPLE16_Averaged_df[(SAMPLE16_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE16_Averaged_df['wavelength'] <= Peak_Max)]
SAMPLE17_Processed_Selectaverage_df = SAMPLE17_Averaged_df[(SAMPLE17_Averaged_df['wavelength'] >= Peak_Min) & (SAMPLE17_Averaged_df['wavelength'] <= Peak_Max)]

SAMPLE1_Processed_Selectaverage_df

In [ ]:
#GAUSSIAN FITTING

SAMPLE1_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE1_Processed_Selectaverage_df)
SAMPLE2_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE2_Processed_Selectaverage_df)
SAMPLE3_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE3_Processed_Selectaverage_df)
SAMPLE4_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE4_Processed_Selectaverage_df)
SAMPLE5_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE5_Processed_Selectaverage_df)
SAMPLE6_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE6_Processed_Selectaverage_df)
SAMPLE7_Fitted_Averagepoints = (SAMPLE7_Processed_Selectaverage_df)
SAMPLE8_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE8_Processed_Selectaverage_df)
SAMPLE9_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE9_Processed_Selectaverage_df)
SAMPLE10_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE10_Processed_Selectaverage_df)
SAMPLE11_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE11_Processed_Selectaverage_df)
SAMPLE12_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE12_Processed_Selectaverage_df)
SAMPLE13_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE13_Processed_Selectaverage_df)
SAMPLE14_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE14_Processed_Selectaverage_df)
SAMPLE15_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE15_Processed_Selectaverage_df)
SAMPLE16_Fitted_Averagepoints = fit_gaussian_profile(SAMPLE16_Processed_Selectaverage_df)
SAMPLE17_Fitted_Averagepoints = (SAMPLE17_Processed_Selectaverage_df)

SAMPLE1_Fitted_Averagepoints


In [ ]:
Processed_Select_df_Plot = figure(title = 'Si 288.157 - BATCH 2 PEAKS CONSIDERED' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

# Processed_Select_df_Plot.line(SAMPLE1_Fitted_Averagepoints.wavelength,SAMPLE1_Fitted_Averagepoints.sample1 , line_width = 2, color ="blue",legend_label="SAMPLE 1")
# Processed_Select_df_Plot.line(SAMPLE1_Processed_Selectaverage_df.wavelength,SAMPLE1_Processed_Selectaverage_df.sample1 , line_width = 2, color ="blue",line_dash="dashed")

# ###################################################################################################################
# Processed_Select_df_Plot.line(SAMPLE2_Fitted_Averagepoints.wavelength,SAMPLE2_Fitted_Averagepoints.sample2 , line_width = 2, color ="orange",legend_label="SAMPLE 2")
# Processed_Select_df_Plot.line(SAMPLE2_Processed_Selectaverage_df.wavelength,SAMPLE2_Processed_Selectaverage_df.sample2 , line_width = 2, color ="orange",line_dash="dashed")

# # ###############################################################################################################
# Processed_Select_df_Plot.line(SAMPLE3_Fitted_Averagepoints.wavelength,SAMPLE3_Fitted_Averagepoints.sample3 , line_width = 2, color ="green",legend_label="SAMPLE 3")
# Processed_Select_df_Plot.line(SAMPLE3_Processed_Selectaverage_df.wavelength,SAMPLE3_Processed_Selectaverage_df.sample3 , line_width = 2, color ="green",line_dash="dashed")

# # # # ######################################################################################################################
# Processed_Select_df_Plot.line(SAMPLE4_Fitted_Averagepoints.wavelength,SAMPLE4_Fitted_Averagepoints.sample4 , line_width = 2, color ="red",legend_label="SAMPLE 4")
# Processed_Select_df_Plot.line(SAMPLE4_Processed_Selectaverage_df.wavelength,SAMPLE4_Processed_Selectaverage_df.sample4 , line_width = 2, color ="red",line_dash="dashed")

# # #############################################################################################################
# Processed_Select_df_Plot.line(SAMPLE5_Fitted_Averagepoints.wavelength,SAMPLE5_Fitted_Averagepoints.sample5 , line_width = 2, color ="purple",legend_label="SAMPLE 5")
# Processed_Select_df_Plot.line(SAMPLE5_Processed_Selectaverage_df.wavelength,SAMPLE5_Processed_Selectaverage_df.sample5 , line_width = 2, color ="purple",line_dash="dashed")

# #############################################################################################################
# Processed_Select_df_Plot.line(SAMPLE6_Fitted_Averagepoints.wavelength,SAMPLE6_Fitted_Averagepoints.sample6 , line_width = 2, color ="brown",legend_label="SAMPLE 6")
# Processed_Select_df_Plot.line(SAMPLE6_Processed_Selectaverage_df.wavelength,SAMPLE6_Processed_Selectaverage_df.sample6 , line_width = 2, color ="brown",line_dash="dashed")

##################################################################################################################
# Processed_Select_df_Plot.line(SAMPLE7_Fitted_Averagepoints.wavelength,SAMPLE7_Fitted_Averagepoints.sample7 , line_width = 2, color ="pink",legend_label="SAMPLE 7")
# Processed_Select_df_Plot.line(SAMPLE7_Processed_Selectaverage_df.wavelength,SAMPLE7_Processed_Selectaverage_df.sample7 , line_width = 2, color ="pink",line_dash="dashed")

# # #########################################################################################################
# Processed_Select_df_Plot.line(SAMPLE8_Fitted_Averagepoints.wavelength,SAMPLE8_Fitted_Averagepoints.sample8 , line_width = 2, color ="gray",legend_label="SAMPLE 8")
# Processed_Select_df_Plot.line(SAMPLE8_Processed_Selectaverage_df.wavelength,SAMPLE8_Processed_Selectaverage_df.sample8 , line_width = 2, color ="gray",line_dash="dashed")

#########################################################################################################
Processed_Select_df_Plot.line(SAMPLE9_Fitted_Averagepoints.wavelength,SAMPLE9_Fitted_Averagepoints.sample9 , line_width = 2, color ="olive",legend_label="SAMPLE 9")
Processed_Select_df_Plot.line(SAMPLE9_Processed_Selectaverage_df.wavelength,SAMPLE9_Processed_Selectaverage_df.sample9 , line_width = 2, color ="olive",line_dash="dashed")

# # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE10_Fitted_Averagepoints.wavelength,SAMPLE10_Fitted_Averagepoints.sample10 , line_width = 2, color ="Cyan",legend_label="SAMPLE 10")
Processed_Select_df_Plot.line(SAMPLE10_Processed_Selectaverage_df.wavelength,SAMPLE10_Processed_Selectaverage_df.sample10 , line_width = 2, color ="Cyan",line_dash="dashed")

# # # # # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE11_Fitted_Averagepoints.wavelength,SAMPLE11_Fitted_Averagepoints.sample11 , line_width = 2, color ="black",legend_label="SAMPLE 11")
Processed_Select_df_Plot.line(SAMPLE11_Processed_Selectaverage_df.wavelength,SAMPLE11_Processed_Selectaverage_df.sample11 , line_width = 2, color ="black",line_dash="dashed")

# # # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE12_Fitted_Averagepoints.wavelength,SAMPLE12_Fitted_Averagepoints.sample12 , line_width = 2, color ="tomato",legend_label="SAMPLE 12")
Processed_Select_df_Plot.line(SAMPLE12_Processed_Selectaverage_df.wavelength,SAMPLE12_Processed_Selectaverage_df.sample12 , line_width = 2, color ="tomato",line_dash="dashed")

# # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE13_Fitted_Averagepoints.wavelength,SAMPLE13_Fitted_Averagepoints.sample13 , line_width = 2, color ="steelblue",legend_label="SAMPLE 13")
Processed_Select_df_Plot.line(SAMPLE13_Processed_Selectaverage_df.wavelength,SAMPLE13_Processed_Selectaverage_df.sample13 , line_width = 2, color ="steelblue",line_dash="dashed")

# # # ##################################################################################################################
Processed_Select_df_Plot.line(SAMPLE14_Fitted_Averagepoints.wavelength,SAMPLE14_Fitted_Averagepoints.sample14 , line_width = 2, color ="limegreen",legend_label="SAMPLE 14")
Processed_Select_df_Plot.line(SAMPLE14_Processed_Selectaverage_df.wavelength,SAMPLE14_Processed_Selectaverage_df.sample14 , line_width = 2, color ="limegreen",line_dash="dashed")

# # # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE15_Fitted_Averagepoints.wavelength,SAMPLE15_Fitted_Averagepoints.sample15 , line_width = 2, color ="deeppink",legend_label="SAMPLE 15")
Processed_Select_df_Plot.line(SAMPLE15_Processed_Selectaverage_df.wavelength,SAMPLE15_Processed_Selectaverage_df.sample15 , line_width = 2, color ="deeppink",line_dash="dashed")

# # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE16_Fitted_Averagepoints.wavelength,SAMPLE16_Fitted_Averagepoints.sample16 , line_width = 2, color ="gold",legend_label="SAMPLE 16")
Processed_Select_df_Plot.line(SAMPLE16_Processed_Selectaverage_df.wavelength,SAMPLE16_Processed_Selectaverage_df.sample16 , line_width = 2, color ="gold",line_dash="dashed")

# # # # # # # # # # # ###################################################################################################################
Processed_Select_df_Plot.line(SAMPLE17_Fitted_Averagepoints.wavelength,SAMPLE17_Fitted_Averagepoints.sample17 , line_width = 2, color ="indigo",legend_label="SAMPLE 17")
Processed_Select_df_Plot.line(SAMPLE17_Processed_Selectaverage_df.wavelength,SAMPLE17_Processed_Selectaverage_df.sample17 , line_width = 2, color ="indigo",line_dash="dashed")

# # # #################################################################################################################

Processed_Select_df_Plot.width = 500
Processed_Select_df_Plot.height = 550

show(Processed_Select_df_Plot)

# Extracting Peak Heights 

As you see from the Pic above , even after normalizing the Spectra , there are few variations in the Intensities , Before passing them to regression step , the important step would be to do Outlier Removal of the Intensities

In [ ]:
def clean_intensities(processed_df, threshold):
    all_intensities = []

    for col in processed_df.columns[1:]: #PEAK EXTRACTION
        max_intensity = processed_df[col].max()
        min_intensity = processed_df[col].min()
        all_intensities.append(max_intensity)

    all_intensities = np.array(all_intensities)
    mean_intensities = np.mean(all_intensities)
    std_intensities = np.std(all_intensities)
    z_scores = np.abs((all_intensities - mean_intensities) / std_intensities)

    cleaned_intensities = [intensity for intensity, z_score in zip(all_intensities, z_scores) if z_score < threshold]

    return cleaned_intensities ,all_intensities

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# def calculate_peak_area_fwhm(signal, x_values):
#     """Calculate the area under the peak between the Full Width at Half Maximum (FWHM)."""
#     # Find the maximum value of the signal
#     max_value = np.max(signal)
#     half_max = max_value / 2
    
#     # Find indices where the signal crosses half the maximum value
#     above_half_max = signal >= half_max
#     crossing_points = np.where(above_half_max)[0]
    
#     if len(crossing_points) < 2:
#         # Not enough points to define FWHM
#         return 0, None, None

#     # Determine FWHM indices
#     fwhm_start_idx = crossing_points[0]
#     fwhm_end_idx = crossing_points[-1]
    
#     # Extract the portion of the signal between the FWHM points
#     fwhm_signal = signal[fwhm_start_idx:fwhm_end_idx+1]
#     fwhm_x_values = x_values[fwhm_start_idx:fwhm_end_idx+1]
    
#     # Calculate the area under the curve using the trapezoidal rule
#     peak_area = np.trapz(fwhm_signal, fwhm_x_values)
    
#     return peak_area, fwhm_x_values, fwhm_signal

# def clean_intensities(processed_df, threshold):
#     all_areas = []

#     # first column is not part of the signal data (e.g., time or index)
#     x_values = processed_df.iloc[:, 0].values

#     for col in processed_df.columns[1:]:
#         signal = processed_df[col].values
#         peak_area, fwhm_x_values, fwhm_signal = calculate_peak_area_fwhm(signal, x_values)
#         all_areas.append(peak_area)

#         # Plotting the signal and FWHM
#         # plt.figure(figsize=(10, 6))
#         # plt.plot(x_values, signal, label='Signal')
#         # if fwhm_x_values is not None and fwhm_signal is not None:
#         #     plt.fill_between(fwhm_x_values, fwhm_signal, alpha=0.3, label='FWHM Area')
#         #     plt.axhline(y=np.max(signal) / 2, color='r', linestyle='--', label='FWHM Half-Max')
#         # plt.xlabel('Wavelength')
#         # plt.ylabel('Intensity')
#         # plt.title(f'Signal and FWHM Area for {col}')
#         # plt.legend()
#         # # plt.show()

#     all_areas = np.array(all_areas)
#     mean_areas = np.mean(all_areas)
#     std_areas = np.std(all_areas)
#     z_scores = np.abs((all_areas - mean_areas) / std_areas)

#     cleaned_areas = [area for area, z_score in zip(all_areas, z_scores) if z_score < threshold]

#     return cleaned_areas, all_areas

# # Example usage:
# # processed_df = pd.DataFrame({'Time': time_data, 'Signal1': signal1_data, 'Signal2': signal2_data, ...})
# # cleaned_areas, all_areas = clean_intensities(processed_df, threshold=2)


In [ ]:
SAMPLE1_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE1_Fitted_Allpoints, threshold=3)[0]
SAMPLE2_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE2_Fitted_Allpoints, threshold=3)[0]
SAMPLE3_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE3_Fitted_Allpoints, threshold=3)[0]
SAMPLE4_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE4_Fitted_Allpoints, threshold=3)[0]
SAMPLE5_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE5_Fitted_Allpoints, threshold=3)[0]
SAMPLE6_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE6_Fitted_Allpoints, threshold=3)[0]
SAMPLE7_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE7_Fitted_Allpoints, threshold=3)[0]
SAMPLE8_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE8_Fitted_Allpoints, threshold=3)[0]
SAMPLE9_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE9_Fitted_Allpoints, threshold=3)[0]
SAMPLE10_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE10_Fitted_Allpoints, threshold=3)[0]
SAMPLE11_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE11_Fitted_Allpoints, threshold=3)[0]
SAMPLE12_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE12_Fitted_Allpoints, threshold=3)[0]
SAMPLE13_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE13_Fitted_Allpoints, threshold=3)[0]
SAMPLE14_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE14_Fitted_Allpoints, threshold=3)[0]
SAMPLE15_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE15_Fitted_Allpoints, threshold=3)[0]
SAMPLE16_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE16_Fitted_Allpoints, threshold=3)[0]
SAMPLE17_Selectedall_Intensities = clean_intensities(processed_df=SAMPLE17_Fitted_Allpoints, threshold=3)[0]

print(SAMPLE1_Selectedall_Intensities)
print(SAMPLE2_Selectedall_Intensities)
print(SAMPLE3_Selectedall_Intensities)
print(SAMPLE4_Selectedall_Intensities)
print(SAMPLE5_Selectedall_Intensities)
print(SAMPLE6_Selectedall_Intensities)
print(SAMPLE7_Selectedall_Intensities)
print(SAMPLE8_Selectedall_Intensities)
print(SAMPLE9_Selectedall_Intensities)
print(SAMPLE10_Selectedall_Intensities)
print(SAMPLE11_Selectedall_Intensities)
print(SAMPLE12_Selectedall_Intensities)
print(SAMPLE13_Selectedall_Intensities)
print(SAMPLE14_Selectedall_Intensities)
print(SAMPLE15_Selectedall_Intensities)
print(SAMPLE16_Selectedall_Intensities)
print(SAMPLE17_Selectedall_Intensities)

In [ ]:
SAMPLE1_Selected_Intensities = clean_intensities(processed_df=SAMPLE1_Fitted_Averagepoints, threshold=3)[1]
SAMPLE2_Selected_Intensities = clean_intensities(processed_df=SAMPLE2_Fitted_Averagepoints, threshold=3)[1]
SAMPLE3_Selected_Intensities = clean_intensities(processed_df=SAMPLE3_Fitted_Averagepoints, threshold=3)[1]
SAMPLE4_Selected_Intensities = clean_intensities(processed_df=SAMPLE4_Fitted_Averagepoints, threshold=3)[1]
SAMPLE5_Selected_Intensities = clean_intensities(processed_df=SAMPLE5_Fitted_Averagepoints, threshold=3)[1]
SAMPLE6_Selected_Intensities = clean_intensities(processed_df=SAMPLE6_Fitted_Averagepoints, threshold=3)[1]
SAMPLE7_Selected_Intensities = clean_intensities(processed_df=SAMPLE7_Fitted_Averagepoints, threshold=3)[1]
SAMPLE8_Selected_Intensities = clean_intensities(processed_df=SAMPLE8_Fitted_Averagepoints, threshold=3)[1]
SAMPLE9_Selected_Intensities = clean_intensities(processed_df=SAMPLE9_Fitted_Averagepoints, threshold=3)[1]
SAMPLE10_Selected_Intensities = clean_intensities(processed_df=SAMPLE10_Fitted_Averagepoints, threshold=3)[1]
SAMPLE11_Selected_Intensities = clean_intensities(processed_df=SAMPLE11_Fitted_Averagepoints, threshold=3)[1]
SAMPLE12_Selected_Intensities = clean_intensities(processed_df=SAMPLE12_Fitted_Averagepoints, threshold=3)[1]
SAMPLE13_Selected_Intensities = clean_intensities(processed_df=SAMPLE13_Fitted_Averagepoints, threshold=3)[1]
SAMPLE14_Selected_Intensities = clean_intensities(processed_df=SAMPLE14_Fitted_Averagepoints, threshold=3)[1]
SAMPLE15_Selected_Intensities = clean_intensities(processed_df=SAMPLE15_Fitted_Averagepoints, threshold=3)[1]
SAMPLE16_Selected_Intensities = clean_intensities(processed_df=SAMPLE16_Fitted_Averagepoints, threshold=3)[1]
SAMPLE17_Selected_Intensities = clean_intensities(processed_df=SAMPLE17_Fitted_Averagepoints, threshold=3)[1]


print(SAMPLE1_Selected_Intensities)
print(SAMPLE2_Selected_Intensities)
print(SAMPLE3_Selected_Intensities)
print(SAMPLE4_Selected_Intensities)
print(SAMPLE5_Selected_Intensities)
print(SAMPLE6_Selected_Intensities)
print(SAMPLE7_Selected_Intensities)
print(SAMPLE8_Selected_Intensities)
print(SAMPLE9_Selected_Intensities)
print(SAMPLE10_Selected_Intensities)
print(SAMPLE11_Selected_Intensities)
print(SAMPLE12_Selected_Intensities)
print(SAMPLE13_Selected_Intensities)
print(SAMPLE14_Selected_Intensities)
print(SAMPLE15_Selected_Intensities)
print(SAMPLE16_Selected_Intensities)
print(SAMPLE17_Selected_Intensities)

# RMS CALCULATION & CHOOSING DECISION

In [ ]:

def calculate_rms_of_noise(df):
    # Filter the rows where wavelength is less than 215
    noise_df = df[df['wavelength'] < 215]
    
    # Extract the column with measurements (assuming it's the second column)
    measurement_column = noise_df.columns[1]
    noise_values = noise_df[measurement_column]
    
    # Calculate the RMS
    rms = np.sqrt(np.mean(np.square(noise_values)))
    
    return rms


In [ ]:
SAMPLE1_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE1_Averaged_full_df)
SAMPLE2_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE2_Averaged_full_df)
SAMPLE3_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE3_Averaged_full_df)
SAMPLE4_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE4_Averaged_full_df)
SAMPLE5_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE5_Averaged_full_df)
SAMPLE6_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE6_Averaged_full_df)
SAMPLE7_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE7_Averaged_full_df)
SAMPLE8_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE8_Averaged_full_df)
SAMPLE9_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE9_Averaged_full_df)
SAMPLE10_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE10_Averaged_full_df)
SAMPLE11_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE11_Averaged_full_df)
SAMPLE12_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE12_Averaged_full_df)
SAMPLE13_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE13_Averaged_full_df)
SAMPLE14_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE14_Averaged_full_df)
SAMPLE15_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE15_Averaged_full_df)
SAMPLE16_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE16_Averaged_full_df)
SAMPLE17_Averaged_NoiseRMS = calculate_rms_of_noise(SAMPLE17_Averaged_full_df)

SAMPLE1_Averaged_NoiseRMS

In [ ]:
Noise_Threshold = 6

if SAMPLE1_Selected_Intensities <= Noise_Threshold * (SAMPLE1_Averaged_NoiseRMS):
    print ("SAMPLE 1 NOT SELECTED")

if SAMPLE2_Selected_Intensities <= Noise_Threshold * (SAMPLE2_Averaged_NoiseRMS):
    print("SAMPLE 2 NOT SELECTED")

if SAMPLE3_Selected_Intensities <= Noise_Threshold * (SAMPLE3_Averaged_NoiseRMS):
    print("SAMPLE 3 NOT SELECTED")

if SAMPLE4_Selected_Intensities <= Noise_Threshold * (SAMPLE4_Averaged_NoiseRMS):
    print("SAMPLE 4 NOT SELECTED")

if SAMPLE5_Selected_Intensities <= Noise_Threshold * (SAMPLE5_Averaged_NoiseRMS):
    print("SAMPLE 5 NOT SELECTED")

if SAMPLE6_Selected_Intensities <= Noise_Threshold * (SAMPLE6_Averaged_NoiseRMS):
    print("SAMPLE 6 NOT SELECTED")

if SAMPLE7_Selected_Intensities <= Noise_Threshold * (SAMPLE7_Averaged_NoiseRMS):
    print("SAMPLE 7 NOT SELECTED")

if SAMPLE8_Selected_Intensities <= Noise_Threshold * (SAMPLE8_Averaged_NoiseRMS):
    print("SAMPLE 8 NOT SELECTED")

if SAMPLE9_Selected_Intensities <= Noise_Threshold * (SAMPLE9_Averaged_NoiseRMS):
    print("SAMPLE 9 NOT SELECTED")

if SAMPLE10_Selected_Intensities <= Noise_Threshold * (SAMPLE10_Averaged_NoiseRMS):
    print("SAMPLE 10 NOT SELECTED")

if SAMPLE11_Selected_Intensities <= Noise_Threshold * (SAMPLE11_Averaged_NoiseRMS):
    print("SAMPLE 11 NOT SELECTED")

if SAMPLE12_Selected_Intensities <= Noise_Threshold * (SAMPLE12_Averaged_NoiseRMS):
    print("SAMPLE 12 NOT SELECTED")

if SAMPLE13_Selected_Intensities <= Noise_Threshold * (SAMPLE13_Averaged_NoiseRMS):
    print("SAMPLE 13 NOT SELECTED")

if SAMPLE14_Selected_Intensities <= Noise_Threshold * (SAMPLE14_Averaged_NoiseRMS):
    print("SAMPLE 14 NOT SELECTED")

if SAMPLE15_Selected_Intensities <= Noise_Threshold * (SAMPLE15_Averaged_NoiseRMS):
    print("SAMPLE 15 NOT SELECTED")

if SAMPLE16_Selected_Intensities <= Noise_Threshold * (SAMPLE16_Averaged_NoiseRMS):
    print("SAMPLE 16 NOT SELECTED")

if SAMPLE17_Selected_Intensities <= Noise_Threshold * (SAMPLE17_Averaged_NoiseRMS):
    print("SAMPLE 17 NOT SELECTED")

# REGRESSION

At the last , now we have list we have selected intensities for each sample , also we have the concentration values per element of each sample , we can now do the regression

In [ ]:
# Regression for C , Mn , Si , Al ,Cr ,Cu , Ni , Nb ,Mo ,Ti (BATCH 2 and 3 COMBINED)

In [ ]:
intensity_data = {
    'Intensity': [
        SAMPLE1_Selectedall_Intensities,
        SAMPLE2_Selectedall_Intensities,
        SAMPLE3_Selectedall_Intensities,
        SAMPLE4_Selectedall_Intensities,
        SAMPLE5_Selectedall_Intensities,
        SAMPLE6_Selectedall_Intensities,
        SAMPLE7_Selectedall_Intensities,
        SAMPLE8_Selectedall_Intensities,
        SAMPLE9_Selectedall_Intensities,
        SAMPLE10_Selectedall_Intensities,
        SAMPLE11_Selectedall_Intensities,
        SAMPLE12_Selectedall_Intensities,
        SAMPLE13_Selectedall_Intensities,
        SAMPLE14_Selectedall_Intensities,
        SAMPLE15_Selectedall_Intensities,
        SAMPLE16_Selectedall_Intensities,
        SAMPLE17_Selectedall_Intensities
    ],
    
    'C_concentration':  [
    0.15, 0.15, 0.16, 0.23, 0.21, 0.07, 0.003, 0.06, 
    0.1222, 0.0847, 0.2990, 0.0032, 0.2734, 0.8027, 0.6353, 0.6632, 0.0943
    ],

    'Mn_concentration':  [
    1.9, 1.8, 2.3, 1.2, 1.9, 2.3, 0.1, 1.1,
    0.571, 0.489, 0.622, 0.000, 0.347, 0.428, 0.473, 0.515, 0.208
    ],

    'Si_concentration':  [
    0.3, 0.3, 0.5, 0.2, 1.8, 0.2, 0.0, 0.4,
    0.13, 0.10, 0.32, 0.00, 0.09, 0.67, 0.58, 0.33, 0.00
    ],

    'Al_concentration':  [
    0.04, 0.04, 0.69, 0.03, 0.04, 0.04, 0.02, 0.04,
    0.280, 0.413, 3.430, 0.000, 0.000, 0.775, 1.995, 0.029, 0.000
    ],

    'Cr_concentration':  [
    0.41, 0.42, 0.70, 0.14, 0.05, 0.60, 0.03, 0.03,
    0.550, 0.020, 2.150, 0.000, 0.021, 0.591, 0.184, 0.194, 0.007
    ],

    'Cu_concentration':  [
    0.02, 0.02, 0.03, 0.02, 0.03, 0.02, 0.01, 0.01,
    0.036, 0.101, 0.049, 0.006, 0.044, 0.220, 0.430, 1.585, 0.067
    ],
    'Ni_concentration':  [
    0.030, 0.037, 0.048, 0.020, 0.037, 0.040, 0.020, 0.010,
    0.000, 0.000, 0.000, 0.000, 0.047, 0.054, 0.075, 0.077, 0.023
    ],
    'Nb_concentration':  [
    0.006, 0.006, 0.032, 0.001, 0.056, 0.001, 0.001, 0.046,
    0.0191, 0.0000, 0.0337, 0.0000, 0.0021, 0.0169, 0.0136, 0.0000, 0.0030
    ],
    'Mo_concentration':  [
    0.101, 0.105, 0.023, 0.000, 0.006, 0.020, 0.005, 0.002,
    0.000, 0.007, 0.126, 0.007, 0.008, 0.010, 0.005, 0.021, 0.007
    ],
    'Ti_concentration':  [
    0.044, 0.037, 0.025, 0.034, 0.014, 0.051, 0.068, 0.028,
    0.0353, 0.0086, 0.0502, 0.0000, 0.0000, 0.0181, 0.0158, 0.0067, 0.0000
    ]
    }

# Creating the DataFrame
calibration_dataframe = pd.DataFrame(intensity_data)

# Print the DataFrame
print(calibration_dataframe)


In [ ]:
calibration_dataframe_np = calibration_dataframe.to_numpy()
calibration_dataframe_np.shape

In [ ]:
# Extracting first 8 rows for training

sample_indices_batch1 = [ 1,2 ,3 ,4,5,6,7,8] #Actual Samples to Include

indices_batch1 = [x - 1 for x in sample_indices_batch1]


X_train_1 = calibration_dataframe['Intensity'].values[indices_batch1]
y_train_1 = calibration_dataframe['Mn_concentration'].values[indices_batch1]


In [ ]:
# Extracting last 9  rows for training
sample_indices_batch2 = [9 ,10,11,12 ,13,14,15,16,17] #Actual Samples to Include
indices_batch2 = [y - 1 for y in sample_indices_batch2]


X_train_2 = calibration_dataframe['Intensity'].values[indices_batch2]
y_train_2 = calibration_dataframe['Mn_concentration'].values[indices_batch2]



In [ ]:
# Extracting last 9  rows for training
sample_indices_batch3 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] #Actual Samples to Include
indices_batch3 = [z - 1 for z in sample_indices_batch3]


X_train_3 = calibration_dataframe['Intensity'].values[indices_batch3]
y_train_3 = calibration_dataframe['Mn_concentration'].values[indices_batch3]

# BATCH 2 - CALIBRATION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt_1
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error



# Flatten X_train to fit the model
mean_intensity_1 = np.array([np.mean(sample_1) for sample_1 in X_train_1])
std_dev_intensity_1 = np.array([np.std(sample_1) for sample_1 in X_train_1])

print (mean_intensity_1)

X_flattened_1_x1 = [mean_intensity_1[0]] 
X_flattened_1_x2 = [mean_intensity_1[1]] 
X_flattened_1_x3 = [mean_intensity_1[2]] 
X_flattened_1_x4 = [mean_intensity_1[3]] 
X_flattened_1_x5 = [mean_intensity_1[4]] 
X_flattened_1_x6 = [mean_intensity_1[5]] 
X_flattened_1_x7 = [mean_intensity_1[6]] 
X_flattened_1_x8 = [mean_intensity_1[7]] 


Y_flattened_1_y1 = [y_train_1[0]] 
Y_flattened_1_y2 = [y_train_1[1]] 
Y_flattened_1_y3 = [y_train_1[2]] 
Y_flattened_1_y4 = [y_train_1[3]] 
Y_flattened_1_y5 = [y_train_1[4]] 
Y_flattened_1_y6 = [y_train_1[5]] 
Y_flattened_1_y7 = [y_train_1[6]] 
Y_flattened_1_y8 = [y_train_1[7]] 



X_flattened_1 = np.concatenate([
                    np.array(X_flattened_1_x1).reshape(-1,1), 
                    np.array(X_flattened_1_x2).reshape(-1,1),
                    np.array(X_flattened_1_x3).reshape(-1,1),
                    np.array(X_flattened_1_x4).reshape(-1,1),
                    np.array(X_flattened_1_x5).reshape(-1,1),
                    np.array(X_flattened_1_x6).reshape(-1,1),
                    np.array(X_flattened_1_x7).reshape(-1,1),
                    np.array(X_flattened_1_x8).reshape(-1,1)
                    ])

Y_flattened_1 = np.concatenate([
                    np.array(Y_flattened_1_y1).reshape(-1,1), 
                    np.array(Y_flattened_1_y2).reshape(-1,1),
                    np.array(Y_flattened_1_y3).reshape(-1,1),
                    np.array(Y_flattened_1_y4).reshape(-1,1),
                    np.array(Y_flattened_1_y5).reshape(-1,1),
                    np.array(Y_flattened_1_y6).reshape(-1,1),
                    np.array(Y_flattened_1_y7).reshape(-1,1),
                    np.array(Y_flattened_1_y8).reshape(-1,1)
                    ])

# Initialize the Linear Regression model
model_1 = LinearRegression()

# Fit the model
model_1.fit(X_flattened_1, Y_flattened_1)


# Compute R2 and RMSE
r2_1 = model_1.score(X_flattened_1,Y_flattened_1)
rmse_1 = np.sqrt(mean_squared_error(X_flattened_1,Y_flattened_1))

# Get the model coefficients and intercept
coefficients_1 = model_1.coef_
intercept_1 = model_1.intercept_


# # Plotting with horizontal error bars
plt_1.figure(figsize=(8, 5))

plt_1.errorbar(X_flattened_1_x1,Y_flattened_1_y1 , xerr=std_dev_intensity_1[0], fmt='o', color='blue', ecolor='black', capsize=3,label='SAMPLE 1')
plt_1.errorbar(X_flattened_1_x2,Y_flattened_1_y2 , xerr=std_dev_intensity_1[1], fmt='o', color='orange', ecolor='black', capsize=3,label='SAMPLE 2')
plt_1.errorbar(X_flattened_1_x3,Y_flattened_1_y3 , xerr=std_dev_intensity_1[2], fmt='o', color='green', ecolor='black', capsize=3,label='SAMPLE 3')
plt_1.errorbar(X_flattened_1_x4,Y_flattened_1_y4 , xerr=std_dev_intensity_1[3], fmt='o', color='red', ecolor='black', capsize=3,label='SAMPLE 4')
plt_1.errorbar(X_flattened_1_x5,Y_flattened_1_y5 , xerr=std_dev_intensity_1[4], fmt='o', color='purple', ecolor='black', capsize=3,label='SAMPLE 5')
plt_1.errorbar(X_flattened_1_x6,Y_flattened_1_y6 , xerr=std_dev_intensity_1[5], fmt='o', color='brown', ecolor='black', capsize=3,label='SAMPLE 6')
plt_1.errorbar(X_flattened_1_x7,Y_flattened_1_y7 , xerr=std_dev_intensity_1[6], fmt='o', color='pink', ecolor='black', capsize=3,label='SAMPLE 7')
plt_1.errorbar(X_flattened_1_x8,Y_flattened_1_y8 , xerr=std_dev_intensity_1[7], fmt='o', color='gray', ecolor='black', capsize=3,label='SAMPLE 8')

plt_1.plot(X_flattened_1,model_1.predict(X_flattened_1), color ='black' , linewidth = 0.5)


title_1 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 2 REFERENCE'
plt_1.xlabel('Intensities (a.u.)')
plt_1.ylabel('Concentration (%) ')
plt_1.title(title_1)
plt_1.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt_1.text(0.85, 0.9, f'y = {coefficients_1:}x + {intercept_1:}', transform=plt.gcf().transFigure, fontsize=10)
plt_1.text(0.92,0.05, f'R^2 = {r2_1:.2f}\nRMSE = {rmse_1:.2f}', transform=plt.gcf().transFigure, fontsize=10)
plt_1.savefig((title_1)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_1.show()


# BATCH 2 VALIDATION


In [ ]:

import matplotlib.pyplot as plt_4


X_validation_1_x1 = [mean_intensity_1[0]] 
X_validation_1_x2 = [mean_intensity_1[1]] 
X_validation_1_x3 = [mean_intensity_1[2]] 
X_validation_1_x4 = [mean_intensity_1[3]] 
X_validation_1_x5 = [mean_intensity_1[4]] 
X_validation_1_x6 = [mean_intensity_1[5]] 
X_validation_1_x7 = [mean_intensity_1[6]] 
X_validation_1_x8 = [mean_intensity_1[7]] 



Y_validation_1_y1  =[y_train_1[0]] 
Y_validation_1_y2 = [y_train_1[1]] 
Y_validation_1_y3 = [y_train_1[2]] 
Y_validation_1_y4 = [y_train_1[3]] 
Y_validation_1_y5 = [y_train_1[4]] 
Y_validation_1_y6 = [y_train_1[5]] 
Y_validation_1_y7 = [y_train_1[6]] 
Y_validation_1_y8 = [y_train_1[7]] 


X_validation_1 = np.concatenate([
                    np.array(X_validation_1_x1).reshape(-1,1), 
                    np.array(X_validation_1_x2).reshape(-1,1),
                    np.array(X_validation_1_x3).reshape(-1,1),
                    np.array(X_validation_1_x4).reshape(-1,1),
                    #np.array(X_validation_1_x5).reshape(-1,1),
                    np.array(X_validation_1_x6).reshape(-1,1)
                    #np.array(X_validation_1_x7).reshape(-1,1),
                    #np.array(X_validation_1_x8).reshape(-1,1)
                    ])



Y_validation_1 = np.concatenate([
                    np.array(Y_validation_1_y1).reshape(-1,1), 
                    np.array(Y_validation_1_y2).reshape(-1,1),
                    np.array(Y_validation_1_y3).reshape(-1,1),
                    np.array(Y_validation_1_y4).reshape(-1,1),
                    #np.array(Y_validation_1_y5).reshape(-1,1),
                    np.array(Y_validation_1_y6).reshape(-1,1)
                    #np.array(Y_validation_1_y7).reshape(-1,1),
                    #np.array(Y_validation_1_y8).reshape(-1,1)
                    ])

Y_validationall_1 = np.concatenate([
                    np.array(Y_validation_1_y1).reshape(-1,1), 
                    np.array(Y_validation_1_y2).reshape(-1,1),
                    np.array(Y_validation_1_y3).reshape(-1,1),
                    np.array(Y_validation_1_y4).reshape(-1,1),
                    np.array(Y_validation_1_y5).reshape(-1,1),
                    np.array(Y_validation_1_y6).reshape(-1,1),
                    np.array(Y_validation_1_y7).reshape(-1,1),
                    np.array(Y_validation_1_y8).reshape(-1,1)
                    ])



Y_predicted_1 = model_1.predict(X_validation_1)

Y_predicted_1_a = np.std(model_1.predict(np.array(X_train_2[1]).reshape(-1,1)))
Y_predicted_1_b = np.std(model_1.predict(np.array(X_train_2[3]).reshape(-1,1)))


r2_val_1 = r2_score(Y_validation_1, Y_predicted_1)
rmse_val_1 = np.sqrt(mean_squared_error(Y_validation_1, Y_predicted_1))

print('R2: ' + str(r2_val_1))
print('RMSE: '+ str(rmse_val_1))

###############################################################

plt_4.figure(figsize=(8, 5))
#plt_4.scatter(Y_validation_1, Y_predicted_1, color='black', label='Predicted vs Actual')

plt_4.errorbar(Y_validation_1[0], Y_predicted_1[0]  , yerr=Y_predicted_1_a, fmt='o', color='orange', ecolor='black', capsize=3,label='SAMPLE 2')
plt_4.errorbar(Y_validation_1[1], Y_predicted_1[1]  , yerr=Y_predicted_1_b, fmt='o', color='red', ecolor='black', capsize=3,label='SAMPLE 4')

# Plot the 45-degree line (y=x) for reference

plt_4.plot([min(Y_validationall_1), max(Y_validationall_1)], 
         [min(Y_validationall_1), max(Y_validationall_1)], 
         color='red', linestyle='--')

title_4 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 2 - VALIDATION RESULTS'

# Add labels and title
plt_4.xlabel('Actual Concentration (%)')
plt_4.ylabel('Predicted Concentration (%)')
plt_4.title(title_4)
plt_4.legend(bbox_to_anchor=(1, 1), loc='upper left')


plt_4.text(0.92,0.05, f'R^2 = {r2_val_1:.4f}\nRMSE = {rmse_val_1:.4f}', transform=plt.gcf().transFigure, fontsize=10)
plt_4.savefig((title_4)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_4.show()

# BATCH 3 - CALIBRATION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt_2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error



# Flatten X_train to fit the model
mean_intensity_2 = np.array([np.mean(sample_2) for sample_2 in X_train_2])
std_dev_intensity_2 = np.array([np.std(sample_2) for sample_2 in X_train_2])

X_flattened_2_x9 =  [mean_intensity_2[0]] 
X_flattened_2_x10 = [mean_intensity_2[1]] 
X_flattened_2_x11 = [mean_intensity_2[2]] 
X_flattened_2_x12 = [mean_intensity_2[3]] 
X_flattened_2_x13 = [mean_intensity_2[4]] 
X_flattened_2_x14 = [mean_intensity_2[5]] 
X_flattened_2_x15 = [mean_intensity_2[6]] 
X_flattened_2_x16 = [mean_intensity_2[7]] 
X_flattened_2_x17 = [mean_intensity_2[8]] 



Y_flattened_2_y9  = [y_train_2[0]] 
Y_flattened_2_y10 = [y_train_2[1]] 
Y_flattened_2_y11 = [y_train_2[2]] 
Y_flattened_2_y12 = [y_train_2[3]] 
Y_flattened_2_y13 = [y_train_2[4]] 
Y_flattened_2_y14 = [y_train_2[5]] 
Y_flattened_2_y15 = [y_train_2[6]] 
Y_flattened_2_y16 = [y_train_2[7]] 
Y_flattened_2_y17 = [y_train_2[8]] 




X_flattened_2 = np.concatenate([
                    np.array(X_flattened_2_x9).reshape(-1,1), 
                    np.array(X_flattened_2_x10).reshape(-1,1),
                    np.array(X_flattened_2_x11).reshape(-1,1),
                    np.array(X_flattened_2_x12).reshape(-1,1),
                    np.array(X_flattened_2_x13).reshape(-1,1),
                    np.array(X_flattened_2_x14).reshape(-1,1),
                    np.array(X_flattened_2_x15).reshape(-1,1),
                    np.array(X_flattened_2_x16).reshape(-1,1),
                    np.array(X_flattened_2_x17).reshape(-1,1)
                    ])

Y_flattened_2 = np.concatenate([
                    np.array(Y_flattened_2_y9).reshape(-1,1), 
                    np.array(Y_flattened_2_y10).reshape(-1,1),
                    np.array(Y_flattened_2_y11).reshape(-1,1),
                    np.array(Y_flattened_2_y12).reshape(-1,1),
                    np.array(Y_flattened_2_y13).reshape(-1,1),
                    np.array(Y_flattened_2_y14).reshape(-1,1),
                    np.array(Y_flattened_2_y15).reshape(-1,1),
                    np.array(Y_flattened_2_y16).reshape(-1,1),
                    np.array(Y_flattened_2_y17).reshape(-1,1)
                    ])


# Initialize the Linear Regression model
model_2 = LinearRegression()

# Fit the model
model_2.fit(X_flattened_2, Y_flattened_2)



# Compute R2 and RMSE
r2_2 = model_2.score(X_flattened_2,Y_flattened_2)
rmse_2 = np.sqrt(mean_squared_error(X_flattened_2,Y_flattened_2))

# Get the model coefficients and intercept
coefficients_2 = model_2.coef_
intercept_2 = model_2.intercept_

print((coefficients_2 * X_flattened_2_x14) + intercept_2)
    
# # Plotting with horizontal error bars
plt_2.figure(figsize=(8, 5))

plt_2.errorbar(X_flattened_2_x9, Y_flattened_2_y9  , xerr=std_dev_intensity_2[0], fmt='o', color='olive', ecolor='black', capsize=3,label='SAMPLE 9')
plt_2.errorbar(X_flattened_2_x10,Y_flattened_2_y10 , xerr=std_dev_intensity_2[1], fmt='o', color='cyan', ecolor='black', capsize=3,label='SAMPLE 10')
plt_2.errorbar(X_flattened_2_x11,Y_flattened_2_y11 , xerr=std_dev_intensity_2[2], fmt='o', color='black', ecolor='black', capsize=3,label='SAMPLE 11')
plt_2.errorbar(X_flattened_2_x12,Y_flattened_2_y12 , xerr=std_dev_intensity_2[3], fmt='o', color='tomato', ecolor='black', capsize=3,label='SAMPLE 12')
plt_2.errorbar(X_flattened_2_x13,Y_flattened_2_y13 , xerr=std_dev_intensity_2[4], fmt='o', color='steelblue', ecolor='black', capsize=3,label='SAMPLE 13')
plt_2.errorbar(X_flattened_2_x14,Y_flattened_2_y14 , xerr=std_dev_intensity_2[5], fmt='o', color='limegreen', ecolor='black', capsize=3,label='SAMPLE 14')
plt_2.errorbar(X_flattened_2_x15,Y_flattened_2_y15 , xerr=std_dev_intensity_2[6], fmt='o', color='deeppink', ecolor='black', capsize=3,label='SAMPLE 15')
plt_2.errorbar(X_flattened_2_x16,Y_flattened_2_y16 , xerr=std_dev_intensity_2[7], fmt='o', color='gold', ecolor='black', capsize=3,label='SAMPLE 16')
plt_2.errorbar(X_flattened_2_x17,Y_flattened_2_y17 , xerr=std_dev_intensity_2[8], fmt='o', color='indigo', ecolor='black', capsize=3,label='SAMPLE 17')


plt_2.plot(X_flattened_2,model_2.predict(X_flattened_2), color ='black' , linewidth = 0.5)


title_2 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 3 REFERENCE'
plt_2.xlabel('Intensities (a.u.)')
plt_2.ylabel('Concentration (%) ')
plt_2.title(title_2)
plt_2.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt_2.text(0.85, 0.9, f'y = {coefficients_2:}x + {intercept_2:}', transform=plt.gcf().transFigure, fontsize=10)
plt_2.text(0.92,0.05, f'R^2 = {r2_2:.2f}\nRMSE = {rmse_2:.2f}', transform=plt.gcf().transFigure, fontsize=10)
# plt_2.savefig((title_2)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_2.show()


# BATCH 3 VALIDATION

In [ ]:
import matplotlib.pyplot as plt_5


X_validation_2_x9 =  [mean_intensity_2[0]] 
X_validation_2_x10 = [mean_intensity_2[1]] 
X_validation_2_x11 = [mean_intensity_2[2]] 
X_validation_2_x12 = [mean_intensity_2[3]] 
X_validation_2_x13 = [mean_intensity_2[4]] 
X_validation_2_x14 = [mean_intensity_2[5]] 
X_validation_2_x15 = [mean_intensity_2[6]] 
X_validation_2_x16 = [mean_intensity_2[7]] 
X_validation_2_x17 = [mean_intensity_2[8]] 

Y_validation_2_y9  = [y_train_2[0]] 
Y_validation_2_y10 = [y_train_2[1]] 
Y_validation_2_y11 = [y_train_2[2]] 
Y_validation_2_y12 = [y_train_2[3]] 
Y_validation_2_y13 = [y_train_2[4]] 
Y_validation_2_y14 = [y_train_2[5]] 
Y_validation_2_y15 = [y_train_2[6]] 
Y_validation_2_y16 = [y_train_2[7]] 
Y_validation_2_y17 = [y_train_2[8]] 


X_validation_2 = np.concatenate([
                    np.array(X_validation_2_x9).reshape(-1,1), 
                    np.array(X_validation_2_x10).reshape(-1,1),
                    np.array(X_validation_2_x11).reshape(-1,1),
                    #np.array(X_validation_2_x12).reshape(-1,1),
                    np.array(X_validation_2_x13).reshape(-1,1),
                    #np.array(X_validation_2_x14).reshape(-1,1),
                    #np.array(X_validation_2_x15).reshape(-1,1),
                    #np.array(X_validation_2_x16).reshape(-1,1),
                    np.array(X_validation_2_x17).reshape(-1,1)
                    ])
Y_validation_2 = np.concatenate([
                    np.array(Y_validation_2_y9).reshape(-1,1), 
                    np.array(Y_validation_2_y10).reshape(-1,1),
                    np.array(Y_validation_2_y11).reshape(-1,1),
                    #np.array(Y_validation_2_y12).reshape(-1,1),
                    np.array(Y_validation_2_y13).reshape(-1,1),
                    #np.array(Y_validation_2_y14).reshape(-1,1),
                    #np.array(Y_validation_2_y15).reshape(-1,1),
                    #np.array(Y_validation_2_y16).reshape(-1,1),
                    np.array(Y_validation_2_y17).reshape(-1,1)
                    ])

Y_validationall_2 = np.concatenate([
                    np.array(Y_validation_2_y9).reshape(-1,1), 
                    np.array(Y_validation_2_y10).reshape(-1,1),
                    np.array(Y_validation_2_y11).reshape(-1,1),
                    np.array(Y_validation_2_y12).reshape(-1,1),
                    np.array(Y_validation_2_y13).reshape(-1,1),
                    np.array(Y_validation_2_y14).reshape(-1,1),
                    np.array(Y_validation_2_y15).reshape(-1,1),
                    np.array(Y_validation_2_y16).reshape(-1,1),
                    np.array(Y_validation_2_y17).reshape(-1,1)
                    ])

Y_predicted_2 = model_2.predict(X_validation_2)

Y_predicted_2_a = np.std(model_2.predict(np.array(X_train_2[1]).reshape(-1,1)))
Y_predicted_2_b = np.std(model_2.predict(np.array(X_train_2[5]).reshape(-1,1)))


r2_val_2 = r2_score(Y_validation_2, Y_predicted_2)
rmse_val_2 = np.sqrt(mean_squared_error(Y_validation_2, Y_predicted_2))


print('R2: ' + str(r2_val_2))
print('RMSE: '+ str(rmse_val_2))

##################################################################################

plt_5.figure(figsize=(8, 5))

plt_5.errorbar(Y_validation_2[0], Y_predicted_2[0]  , yerr=Y_predicted_2_a, fmt='o', color='cyan', ecolor='black', capsize=3,label='SAMPLE 10')
plt_5.errorbar(Y_validation_2[1], Y_predicted_2[1]  , yerr=Y_predicted_2_b, fmt='o', color='limegreen', ecolor='black', capsize=3,label='SAMPLE 14')


# Plot the 45-degree line (y=x) for reference
plt_5.plot([min(Y_validationall_2), max(Y_validationall_2)], 
         [min(Y_validationall_2), max(Y_validationall_2)], 
         color='red', linestyle='--')

title_5 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 3 - VALIDATION RESULTS'

# Add labels and title
plt_5.xlabel('Actual Concentration (%)')
plt_5.ylabel('Predicted Concentration(%)')
plt_5.title(title_5)
plt_5.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt_5.text(0.92,0.05, f'R^2 = {r2_val_2:.4f}\nRMSE = {rmse_val_2:.4f}', transform=plt.gcf().transFigure, fontsize=10)

# plt_5.savefig((title_5)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_5.show()


# BATCH 2 and 3 - CALIBRATION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt_3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error



# Flatten X_train to fit the model
mean_intensity_3 = np.array([np.mean(sample_3) for sample_3 in X_train_3])
std_dev_intensity_3 = np.array([np.std(sample_3) for sample_3 in X_train_3])


X_flattened_3_x1 =  [mean_intensity_3[0]] 
X_flattened_3_x2 =  [mean_intensity_3[1]] 
X_flattened_3_x3 =  [mean_intensity_3[2]] 
X_flattened_3_x4 =  [mean_intensity_3[3]] 
X_flattened_3_x5 =  [mean_intensity_3[4]] 
X_flattened_3_x6 =  [mean_intensity_3[5]] 
X_flattened_3_x7 =  [mean_intensity_3[6]] 
X_flattened_3_x8 =  [mean_intensity_3[7]] 
X_flattened_3_x9 =  [mean_intensity_3[8]] 
X_flattened_3_x10 = [mean_intensity_3[9]] 
X_flattened_3_x11 = [mean_intensity_3[10]] 
X_flattened_3_x12 = [mean_intensity_3[11]] 
X_flattened_3_x13 = [mean_intensity_3[12]] 
X_flattened_3_x14 = [mean_intensity_3[13]] 
X_flattened_3_x15 = [mean_intensity_3[14]] 
X_flattened_3_x16 = [mean_intensity_3[15]] 
X_flattened_3_x17 = [mean_intensity_3[16]] 


Y_flattened_3_y1  = [y_train_3[0]] 
Y_flattened_3_y2 =  [y_train_3[1]] 
Y_flattened_3_y3 =  [y_train_3[2]] 
Y_flattened_3_y4 =  [y_train_3[3]] 
Y_flattened_3_y5 =  [y_train_3[4]] 
Y_flattened_3_y6 =  [y_train_3[5]] 
Y_flattened_3_y7 =  [y_train_3[6]] 
Y_flattened_3_y8 =  [y_train_3[7]] 
Y_flattened_3_y9  = [y_train_3[8]] 
Y_flattened_3_y10 = [y_train_3[9]] 
Y_flattened_3_y11 = [y_train_3[10]] 
Y_flattened_3_y12 = [y_train_3[11]] 
Y_flattened_3_y13 = [y_train_3[12]] 
Y_flattened_3_y14 = [y_train_3[13]] 
Y_flattened_3_y15 = [y_train_3[14]] 
Y_flattened_3_y16 = [y_train_3[15]] 
Y_flattened_3_y17 = [y_train_3[16]] 




X_flattened_3 = np.concatenate([
                    np.array(X_flattened_3_x1).reshape(-1,1),
                    np.array(X_flattened_3_x2).reshape(-1,1), 
                    np.array(X_flattened_3_x3).reshape(-1,1), 
                    np.array(X_flattened_3_x4).reshape(-1,1), 
                    np.array(X_flattened_3_x5).reshape(-1,1), 
                    np.array(X_flattened_3_x6).reshape(-1,1),
                    np.array(X_flattened_3_x7).reshape(-1,1), 
                    np.array(X_flattened_3_x8).reshape(-1,1), 
                    np.array(X_flattened_3_x9).reshape(-1,1), 
                    np.array(X_flattened_3_x10).reshape(-1,1),
                    np.array(X_flattened_3_x11).reshape(-1,1),
                    np.array(X_flattened_3_x12).reshape(-1,1),
                    np.array(X_flattened_3_x13).reshape(-1,1),
                    np.array(X_flattened_3_x14).reshape(-1,1),
                    np.array(X_flattened_3_x15).reshape(-1,1),
                    np.array(X_flattened_3_x16).reshape(-1,1),
                    np.array(X_flattened_3_x17).reshape(-1,1)
                    ])

Y_flattened_3 = np.concatenate([
                    np.array(Y_flattened_3_y1).reshape(-1,1), 
                    np.array(Y_flattened_3_y2).reshape(-1,1),
                    np.array(Y_flattened_3_y3).reshape(-1,1),
                    np.array(Y_flattened_3_y4).reshape(-1,1),
                    np.array(Y_flattened_3_y5).reshape(-1,1),
                    np.array(Y_flattened_3_y6).reshape(-1,1),
                    np.array(Y_flattened_3_y7).reshape(-1,1),
                    np.array(Y_flattened_3_y8).reshape(-1,1),
                    np.array(Y_flattened_3_y9).reshape(-1,1), 
                    np.array(Y_flattened_3_y10).reshape(-1,1),
                    np.array(Y_flattened_3_y11).reshape(-1,1),
                    np.array(Y_flattened_3_y12).reshape(-1,1),
                    np.array(Y_flattened_3_y13).reshape(-1,1),
                    np.array(Y_flattened_3_y14).reshape(-1,1),
                    np.array(Y_flattened_3_y15).reshape(-1,1),
                    np.array(Y_flattened_3_y16).reshape(-1,1),
                    np.array(Y_flattened_3_y17).reshape(-1,1)
                    ])


# Initialize the Linear Regression model
model_3 = LinearRegression()

# Fit the model
model_3.fit(X_flattened_3, Y_flattened_3)

# Compute R2 and RMSE
r2_3 = model_3.score(X_flattened_3,Y_flattened_3)
rmse_3 = np.sqrt(mean_squared_error(X_flattened_3,Y_flattened_3))

# Get the model coefficients and intercept
coefficients_3 = model_3.coef_
intercept_3 = model_3.intercept_

print(coefficients_3)


# # Plotting with horizontal error bars
plt_3.figure(figsize=(8, 5))

plt_3.errorbar(X_flattened_3_x1,  Y_flattened_3_y1 ,  xerr=std_dev_intensity_3[0], fmt='o', color='blue', ecolor='black', capsize=3,label='SAMPLE 1')
plt_3.errorbar(X_flattened_3_x2,  Y_flattened_3_y2 ,  xerr=std_dev_intensity_3[1], fmt='o', color='orange', ecolor='black', capsize=3,label='SAMPLE 2')
plt_3.errorbar(X_flattened_3_x3,  Y_flattened_3_y3 ,  xerr=std_dev_intensity_3[2], fmt='o', color='green', ecolor='black', capsize=3,label='SAMPLE 3')
plt_3.errorbar(X_flattened_3_x4,  Y_flattened_3_y4 ,  xerr=std_dev_intensity_3[3], fmt='o', color='red', ecolor='black', capsize=3,label='SAMPLE 4')
plt_3.errorbar(X_flattened_3_x5,  Y_flattened_3_y5 ,  xerr=std_dev_intensity_3[4], fmt='o', color='purple', ecolor='black', capsize=3,label='SAMPLE 5')
plt_3.errorbar(X_flattened_3_x6,  Y_flattened_3_y6 ,  xerr=std_dev_intensity_3[5], fmt='o', color='brown', ecolor='black', capsize=3,label='SAMPLE 6')
plt_3.errorbar(X_flattened_3_x7,  Y_flattened_3_y7 ,  xerr=std_dev_intensity_3[6], fmt='o', color='pink', ecolor='black', capsize=3,label='SAMPLE 7')
plt_3.errorbar(X_flattened_3_x8,  Y_flattened_3_y8 ,  xerr=std_dev_intensity_3[7], fmt='o', color='gray', ecolor='black', capsize=3,label='SAMPLE 8')
plt_3.errorbar(X_flattened_3_x9,  Y_flattened_3_y9 ,  xerr=std_dev_intensity_3[8], fmt='o', color='olive', ecolor='black', capsize=3,label='SAMPLE 9')
plt_3.errorbar(X_flattened_3_x10, Y_flattened_3_y10 , xerr=std_dev_intensity_3[9], fmt='o', color='cyan', ecolor='black', capsize=3,label='SAMPLE 10')
plt_3.errorbar(X_flattened_3_x11, Y_flattened_3_y11 , xerr=std_dev_intensity_3[10], fmt='o', color='black', ecolor='black', capsize=3,label='SAMPLE 11')
plt_3.errorbar(X_flattened_3_x12, Y_flattened_3_y12 , xerr=std_dev_intensity_3[11], fmt='o', color='tomato', ecolor='black', capsize=3,label='SAMPLE 12')
plt_3.errorbar(X_flattened_3_x13, Y_flattened_3_y13 , xerr=std_dev_intensity_3[12], fmt='o', color='steelblue', ecolor='black', capsize=3,label='SAMPLE 13')
plt_3.errorbar(X_flattened_3_x14, Y_flattened_3_y14 , xerr=std_dev_intensity_3[13], fmt='o', color='limegreen', ecolor='black', capsize=3,label='SAMPLE 14')
plt_3.errorbar(X_flattened_3_x15, Y_flattened_3_y15 , xerr=std_dev_intensity_3[14], fmt='o', color='deeppink', ecolor='black', capsize=3,label='SAMPLE 15')
plt_3.errorbar(X_flattened_3_x16, Y_flattened_3_y16 , xerr=std_dev_intensity_3[15], fmt='o', color='gold', ecolor='black', capsize=3,label='SAMPLE 16')
plt_3.errorbar(X_flattened_3_x17, Y_flattened_3_y17 , xerr=std_dev_intensity_3[16], fmt='o', color='indigo', ecolor='black', capsize=3,label='SAMPLE 17')





plt_3.plot(X_flattened_3,model_3.predict(X_flattened_3), color ='black' , linewidth = 0.5)


title_3 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 2 & 3 REFERENCE'
plt_3.xlabel('Intensities (a.u.)')
plt_3.ylabel('Concentration (%) ')
plt_3.title(title_3)
plt_3.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt_3.text(0.85, 0.9, f'y = {coefficients_3:}x + {intercept_3:}', transform=plt.gcf().transFigure, fontsize=10)
plt_3.text(0.92,0.05, f'R^2 = {r2_3:.2f}\nRMSE = {rmse_3:.2f}', transform=plt.gcf().transFigure, fontsize=10)
plt_3.savefig((title_3)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_3.show()


# BATCH 2 and BATCH 3 Validation


In [ ]:
import matplotlib.pyplot as plt_6


X_validation_3_x1 =  [mean_intensity_3[0]] 
X_validation_3_x2 =  [mean_intensity_3[1]] 
X_validation_3_x3 =  [mean_intensity_3[2]] 
X_validation_3_x4 =  [mean_intensity_3[3]] 
X_validation_3_x5 =  [mean_intensity_3[4]] 
X_validation_3_x6 =  [mean_intensity_3[5]] 
X_validation_3_x7 =  [mean_intensity_3[6]] 
X_validation_3_x8 =  [mean_intensity_3[7]] 
X_validation_3_x9 =  [mean_intensity_3[8]] 
X_validation_3_x10 = [mean_intensity_3[9]] 
X_validation_3_x11 = [mean_intensity_3[10]] 
X_validation_3_x12 = [mean_intensity_3[11]] 
X_validation_3_x13 = [mean_intensity_3[12]] 
X_validation_3_x14 = [mean_intensity_3[13]] 
X_validation_3_x15 = [mean_intensity_3[14]] 
X_validation_3_x16 = [mean_intensity_3[15]] 
X_validation_3_x17 = [mean_intensity_3[16]] 


Y_validation_3_y1  = [y_train_3[0]] 
Y_validation_3_y2 =  [y_train_3[1]] 
Y_validation_3_y3 =  [y_train_3[2]] 
Y_validation_3_y4 =  [y_train_3[3]] 
Y_validation_3_y5 =  [y_train_3[4]] 
Y_validation_3_y6 =  [y_train_3[5]] 
Y_validation_3_y7 =  [y_train_3[6]] 
Y_validation_3_y8 =  [y_train_3[7]] 
Y_validation_3_y9  = [y_train_3[8]] 
Y_validation_3_y10 = [y_train_3[9]] 
Y_validation_3_y11 = [y_train_3[10]] 
Y_validation_3_y12 = [y_train_3[11]] 
Y_validation_3_y13 = [y_train_3[12]] 
Y_validation_3_y14 = [y_train_3[13]] 
Y_validation_3_y15 = [y_train_3[14]] 
Y_validation_3_y16 = [y_train_3[15]] 
Y_validation_3_y17 = [y_train_3[16]] 




X_validation_3 = np.concatenate([
                    np.array(X_validation_3_x1).reshape(-1,1),
                    #np.array(X_validation_3_x2).reshape(-1,1), 
                    #np.array(X_validation_3_x3).reshape(-1,1), 
                    #np.array(X_validation_3_x4).reshape(-1,1), 
                    #np.array(X_validation_3_x5).reshape(-1,1), 
                    np.array(X_validation_3_x6).reshape(-1,1),
                    #np.array(X_validation_3_x7).reshape(-1,1), 
                    #np.array(X_validation_3_x8).reshape(-1,1), 
                    #np.array(X_validation_3_x9).reshape(-1,1), 
                    np.array(X_validation_3_x10).reshape(-1,1)
                    #np.array(X_validation_3_x11).reshape(-1,1),
                    #np.array(X_validation_3_x12).reshape(-1,1),
                    #np.array(X_validation_3_x13).reshape(-1,1),
                    #np.array(X_validation_3_x14).reshape(-1,1),
                    #np.array(X_validation_3_x15).reshape(-1,1),
                    #np.array(X_validation_3_x16).reshape(-1,1)
                    #np.array(X_validation_3_x17).reshape(-1,1)
                    ])

Y_validation_3 = np.concatenate([
                    np.array(Y_validation_3_y1).reshape(-1,1), 
                    #np.array(Y_validation_3_y2).reshape(-1,1),
                    #np.array(Y_validation_3_y3).reshape(-1,1),
                    #np.array(Y_validation_3_y4).reshape(-1,1),
                    #np.array(Y_validation_3_y5).reshape(-1,1),
                    np.array(Y_validation_3_y6).reshape(-1,1),
                    #np.array(Y_validation_3_y7).reshape(-1,1),
                    #np.array(Y_validation_3_y8).reshape(-1,1),
                    #np.array(Y_validation_3_y9).reshape(-1,1), 
                    np.array(Y_validation_3_y10).reshape(-1,1)
                    #np.array(Y_validation_3_y11).reshape(-1,1),
                    #np.array(Y_validation_3_y12).reshape(-1,1),
                    #np.array(Y_validation_3_y13).reshape(-1,1),
                    #np.array(Y_validation_3_y14).reshape(-1,1),
                    #np.array(Y_validation_3_y15).reshape(-1,1),
                    #np.array(Y_validation_3_y16).reshape(-1,1)
                    #np.array(Y_validation_3_y17).reshape(-1,1)
                    ])

Y_validationall_3 = np.concatenate([
                    np.array(Y_validation_3_y1).reshape(-1,1), 
                    np.array(Y_validation_3_y2).reshape(-1,1),
                    np.array(Y_validation_3_y3).reshape(-1,1),
                    np.array(Y_validation_3_y4).reshape(-1,1),
                    np.array(Y_validation_3_y5).reshape(-1,1),
                    np.array(Y_validation_3_y6).reshape(-1,1),
                    np.array(Y_validation_3_y7).reshape(-1,1),
                    np.array(Y_validation_3_y8).reshape(-1,1),
                    np.array(Y_validation_3_y9).reshape(-1,1), 
                    np.array(Y_validation_3_y10).reshape(-1,1),
                    np.array(Y_validation_3_y11).reshape(-1,1),
                    np.array(Y_validation_3_y12).reshape(-1,1),
                    np.array(Y_validation_3_y13).reshape(-1,1),
                    np.array(Y_validation_3_y14).reshape(-1,1),
                    np.array(Y_validation_3_y15).reshape(-1,1),
                    np.array(Y_validation_3_y16).reshape(-1,1),
                    np.array(Y_validation_3_y17).reshape(-1,1)
                    ])


Y_predicted_3 = model_3.predict(X_validation_3)

Y_predicted_3_a = np.std(model_3.predict(np.array(X_train_3[0]).reshape(-1,1)))
Y_predicted_3_b = np.std(model_3.predict(np.array(X_train_3[5]).reshape(-1,1)))
Y_predicted_3_c = np.std(model_3.predict(np.array(X_train_3[9]).reshape(-1,1)))



r2_val_3 = r2_score(Y_validation_3, Y_predicted_3)
rmse_val_3 = np.sqrt(mean_squared_error(Y_validation_3, Y_predicted_3))

print('R2: ' + str(r2_val_3))
print('RMSE: '+ str(rmse_val_3))

######################################################

plt_6.figure(figsize=(8, 5))
# plt_6.scatter(Y_validation_3, Y_predicted_3, color='black', label='Predicted vs Actual')


plt_6.errorbar(Y_validation_3[0], Y_predicted_3[0]  , yerr=Y_predicted_3_a, fmt='o', color='blue', ecolor='black', capsize=3,label='SAMPLE 1')
plt_6.errorbar(Y_validation_3[1], Y_predicted_3[1]  , yerr=Y_predicted_3_b, fmt='o', color='brown', ecolor='black', capsize=3,label='SAMPLE 6')
plt_6.errorbar(Y_validation_3[2], Y_predicted_3[2]  , yerr=Y_predicted_3_c, fmt='o', color='cyan', ecolor='black', capsize=3,label='SAMPLE 10')

print(Y_validation_3[2])

# Plot the 45-degree line (y=x) for reference
plt_6.plot([min(Y_validationall_3), max(Y_validationall_3)], 
         [min(Y_validationall_3), max(Y_validationall_3)], 
         color='red', linestyle='--')

title_6 = (Element_name) + ' ' + (Peak_Pos)+ ' - BATCH 2 & 3 - VALIDATION RESULTS'

# Add labels and title
plt_6.xlabel('Actual Concentration (%)')
plt_6.ylabel('Predicted Concentration (%)')
plt_6.title(title_6)
plt_6.legend(bbox_to_anchor=(1, 1), loc='upper left')


plt_6.text(0.92,0.05, f'R^2 = {r2_val_3:.4f}\nRMSE = {rmse_val_3:.4f}', transform=plt.gcf().transFigure, fontsize=10)
# plt_6.savefig((title_6)+ ".png" ,dpi = 1200 , bbox_inches='tight')

plt_6.show()
